In [1]:
# Parameters
input_path = "C:\\Users\\Marco.Africani\\OneDrive - AVU SA\\AVU CPI Campaign\\Puzzle_control_Reports\\SOURCE_FILES"
output_path = "C:\\Users\\Marco.Africani\\OneDrive - AVU SA\\AVU CPI Campaign\\Puzzle_control_Reports\\IRON_DATA"
week_number = 36


In [2]:
# --- CELL 1 AVU_ignition_1.ipynb: Global Setup & Parameter Intake (Papermill/Env/UI-safe) ---
import os
import json
import warnings
from datetime import datetime, date, timedelta
from pathlib import Path
from typing import Any, Dict
import numpy as np
import pandas as pd

warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

# -------------------------------
# 0) Constants used across the run
# -------------------------------
NUM_SLOTS = 5
DAYS_FULL = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
DAYS_LOWER = [d.lower() for d in DAYS_FULL]

# When preferences can't be fully satisfied, later cells can consult these knobs
ENGINE_FILL_POLICY = {
    "min_slots_per_day": NUM_SLOTS,   # aim to fill all visible slots
    "relaxation_enabled": True,
    "relaxation_order": [
        # later cells should progressively relax in this order until enough wines found:
        "loyalty",          # treat 'all' if too strict
        "wine_type",        # allow any type if filter blocks results
        "bottle_size",      # allow any size if specific format too rare
        "price_tier",       # widen tier ranges (Budget→Mid→Premium→Luxury→Ultra)
        "last_stock_only",  # ignore last-stock constraint if needed
        "seasonality"       # ignore seasonality boost as hard constraint (keep as soft weight)
    ],
}

# -------------------------------
# 1) Try to read Papermill parameters (if any)
# -------------------------------
_pm_params: Dict[str, Any] = {}
try:
    from papermill import get_parameters  # type: ignore
    _pm_params = get_parameters() or {}
except Exception:
    _pm_params = {}

def _coerce_int(value, fallback):
    try:
        return int(value)
    except Exception:
        return int(fallback)

def _iso_week_now():
    try:
        return datetime.now().isocalendar().week
    except Exception:
        return int(datetime.now().strftime("%V"))

# -------------------------------
# 2) Resolve week_number (priority: papermill -> env -> runtime default)
# -------------------------------
if "week_number" in globals():
    _week_candidate = globals().get("week_number")
elif "week_number" in _pm_params:
    _week_candidate = _pm_params.get("week_number")
else:
    _week_candidate = os.getenv("WEEK_NUMBER", _iso_week_now())

week_number = _coerce_int(_week_candidate, _iso_week_now())
# Clamp to ISO range
if week_number < 1 or week_number > 53:
    week_number = _iso_week_now()

# Useful week helpers
def week_bounds(iso_week: int, year: int = datetime.now().year):
    # Monday as first day of ISO week
    d = date.fromisocalendar(year, iso_week, 1)
    return d, d + timedelta(days=6)

week_start_date, week_end_date = week_bounds(week_number)

# --- YEAR AWARENESS + EUROPE/ZURICH TZ + PARAM SUPPORT ---
from zoneinfo import ZoneInfo

def _iso_week_now_europe():
    now = datetime.now(ZoneInfo("Europe/Zurich"))
    iso = now.isocalendar()
    return iso.year, iso.week

# Allow papermill/env to pass an explicit calendar year
if "calendar_year" in globals():
    _year_candidate = globals().get("calendar_year")
elif "calendar_year" in _pm_params:
    _year_candidate = _pm_params.get("calendar_year")
else:
    _year_candidate = os.getenv("CALENDAR_YEAR", None)

try:
    calendar_year = int(_year_candidate) if _year_candidate is not None else _iso_week_now_europe()[0]
except Exception:
    calendar_year = _iso_week_now_europe()[0]

# Week bounds should use the chosen calendar_year
def week_bounds(iso_week: int, year: int):
    # Guard invalid 53rd week for some years
    try:
        d = date.fromisocalendar(year, iso_week, 1)
    except ValueError:
        iso_week = min(max(iso_week, 1), 52)
        d = date.fromisocalendar(year, iso_week, 1)
    return d, d + timedelta(days=6)

week_start_date, week_end_date = week_bounds(week_number, calendar_year)

# -------------------------------
# 3) Resolve IO paths (priority: papermill -> env -> sensible defaults)
# -------------------------------
def _resolve_path(param_key, env_key, default_path_str):
    if param_key in globals():
        p = globals().get(param_key)
    elif param_key in _pm_params:
        p = _pm_params.get(param_key)
    else:
        p = os.getenv(env_key, default_path_str)
    return Path(p)

# Defaults match your project structure
_default_source = str(Path.home() / "OneDrive - AVU SA" / "AVU CPI Campaign" / "Puzzle_control_Reports" / "SOURCE_FILES")
_default_output = str(Path.home() / "OneDrive - AVU SA" / "AVU CPI Campaign" / "Puzzle_control_Reports" / "IRON_DATA")

SOURCE_PATH: Path = _resolve_path("input_path", "INPUT_PATH", _default_source)
OUTPUT_PATH: Path = _resolve_path("output_path", "OUTPUT_PATH", _default_output)

# Make sure all dirs exist (incl. subfolders we’ll use)
SOURCE_PATH.mkdir(parents=True, exist_ok=True)
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)
LOCKED_PATH: Path = OUTPUT_PATH / "locked_weeks"
CALENDAR_PATH: Path = OUTPUT_PATH / "calendar"       # where schedule JSONs per week can live
EXPORTS_PATH: Path = OUTPUT_PATH / "exports"         # generic exports
POWERBI_PATH: Path = OUTPUT_PATH / "powerbi"         # enriched Excel for Power BI
TMP_PATH: Path = OUTPUT_PATH / "_tmp"

for p in (LOCKED_PATH, CALENDAR_PATH, EXPORTS_PATH, POWERBI_PATH, TMP_PATH):
    p.mkdir(parents=True, exist_ok=True)

# Optional reports path (kept for compatibility)
REPORTS_PATH: Path = Path.home() / "OneDrive - AVU SA" / "AVU CPI Campaign" / "non_recipient_reports"

# -------------------------------
# 4) Load lightweight UI inputs if present (non-fatal if missing)
#    These files are written by the Flask app before launching the notebook.
# -------------------------------
NOTEBOOKS_DIR = Path("notebooks")
FILTERS_FILE = NOTEBOOKS_DIR / "filters.json"
LOCKED_SNAPSHOT_FILE = NOTEBOOKS_DIR / "locked_calendar.json"  # snapshot of UI locks passed in

def _load_json_or_empty(path: Path):
    try:
        if path.exists() and path.stat().st_size > 0:
            with open(path, "r", encoding="utf-8") as f:
                return json.load(f)
    except Exception:
        pass
    return {}

raw_filters: Dict[str, Any] = _load_json_or_empty(FILTERS_FILE)              # may be {}
_locked_raw = _load_json_or_empty(LOCKED_SNAPSHOT_FILE)                       # may be {}

# -------- Canonicalize UI/legacy locks to one shape --------
# Accepts:
#   - {"weekly_calendar": {day: [slot...]}}
#   - {day: [slot...]}
#   - {day: {"main":[slot...], "overflow":[...]}}
# Returns:
#   {day: {"main": [None-padded to NUM_SLOTS], "overflow": list}}
def _coerce_locked_snapshot(snap, days=None, num_slots=NUM_SLOTS):
    if days is None:
        days = DAYS_FULL

    # unwrap {"weekly_calendar": {...}} if present
    if isinstance(snap, dict) and isinstance(snap.get("weekly_calendar"), dict):
        snap = snap["weekly_calendar"]

    out = {}
    for d in days:
        node = (snap or {}).get(d) or (snap or {}).get(d.lower())
        main, overflow = [], []
        if isinstance(node, list):
            main = node
        elif isinstance(node, dict):
            if isinstance(node.get("main"), list):
                main = node.get("main") or []
            elif isinstance(node.get("slots"), list):
                main = node.get("slots") or []
            else:
                # tolerate indexed dicts {0:...,1:...}
                tmp = []
                for i in range(num_slots):
                    v = node.get(i, node.get(str(i)))
                    tmp.append(v if v else None)
                main = tmp
            if isinstance(node.get("overflow"), list):
                overflow = node.get("overflow") or []
        # clamp & pad main
        main = (main or [])[:num_slots]
        if len(main) < num_slots:
            main = main + [None] * (num_slots - len(main))
        out[d] = {"main": [x if x else None for x in main], "overflow": list(overflow or [])}
    return out

def _load_json(path: Path):
    try:
        if path.exists() and path.stat().st_size > 0:
            return json.loads(path.read_text(encoding="utf-8"))
    except Exception:
        pass
    return None

# Normalize the incoming UI snapshot
LOCKED_SNAPSHOT_NORM = _coerce_locked_snapshot(_locked_raw, DAYS_FULL, NUM_SLOTS)

# If empty, fallback to persisted week file
def _is_empty_main(snap_norm):
    return not any(any(snap_norm[d]["main"]) for d in DAYS_FULL)

if _is_empty_main(LOCKED_SNAPSHOT_NORM):
    persisted = _load_json(LOCKED_PATH / f"locked_calendar_{calendar_year}_week_{week_number}.json")
    LOCKED_SNAPSHOT_NORM = _coerce_locked_snapshot(persisted, DAYS_FULL, NUM_SLOTS)

# Export *both* forms for downstream cells:
# - EFFECTIVE_LOCKS / LOCKED_CALENDAR: {day: [slot,..]} (Cell 6/7 expect this)
# - locked_calendar_snapshot: {day: {"main":[...], "overflow":[...]}} (for any structured readers)
EFFECTIVE_LOCKS = {d: LOCKED_SNAPSHOT_NORM[d]["main"] for d in DAYS_FULL}
LOCKED_CALENDAR = dict(EFFECTIVE_LOCKS)  # alias used by some cells
locked_calendar_snapshot = dict(LOCKED_SNAPSHOT_NORM)  # keep structured version for completeness

# -------------------------------
# 5) Helpers: normalize filters & day keys
# -------------------------------
PRICE_BUCKETS = ["Budget","Mid-range","Premium","Luxury","Ultra Luxury"]

def price_tier_label(price: float) -> str:
    try:
        p = float(price)
    except Exception:
        return ""
    if p < 50: return "Budget"
    elif p < 100: return "Mid-range"
    elif p < 200: return "Premium"
    elif p < 500: return "Luxury"
    else: return "Ultra Luxury"

DEFAULT_FILTERS: Dict[str, Any] = {
    "loyalty": "all",
    "wine_type": None,        # None = all
    "bottle_size": 750,       # ml
    "price_tier_bucket": "",  # "" = all
    "last_stock": False,
    "last_stock_threshold": 10,
    "seasonality_boost": False,
    "style": "default",
    "calendar_day": None
}

def _as_bool(x): 
    if isinstance(x, bool): return x
    if isinstance(x, (int, float)): return bool(x)
    if isinstance(x, str): return x.strip().lower() in {"1","true","yes","y","on"}
    return False

def _as_int(x, default=None):
    try:
        return int(x)
    except Exception:
        return default

def normalize_filters(f: Dict[str, Any]) -> Dict[str, Any]:
    f = dict(f or {})
    out = dict(DEFAULT_FILTERS)

    out["loyalty"] = str(f.get("loyalty", out["loyalty"])).strip().lower() or "all"

    wt = f.get("wine_type", out["wine_type"])
    if isinstance(wt, str) and wt.strip().lower() == "all":
        wt = None
    out["wine_type"] = wt

    out["bottle_size"] = _as_int(f.get("bottle_size", out["bottle_size"]), 750)

    pt = str(f.get("price_tier_bucket", out["price_tier_bucket"])).strip()
    out["price_tier_bucket"] = pt if pt in PRICE_BUCKETS or pt == "" else ""

    out["last_stock"] = _as_bool(f.get("last_stock", out["last_stock"]))
    out["last_stock_threshold"] = _as_int(f.get("last_stock_threshold", out["last_stock_threshold"]), 10)

    out["seasonality_boost"] = _as_bool(f.get("seasonality_boost", out["seasonality_boost"]))

    style = str(f.get("style", out["style"])).strip().lower()
    out["style"] = style if style in {"default","cat","nigo"} else "default"

    cd = f.get("calendar_day", None)
    out["calendar_day"] = str(cd) if cd else None

    return out

filters = normalize_filters(raw_filters)

# -------------------------------
# 6) Echo configuration
# -------------------------------
print(f"✅ Source data path:   {SOURCE_PATH.resolve()}")
print(f"✅ Output data path:   {OUTPUT_PATH.resolve()}")
print(f"📦 Locked weeks path:  {LOCKED_PATH.resolve()}")
print(f"📄 Exports path:       {EXPORTS_PATH.resolve()}")
print(f"📊 PowerBI path:       {POWERBI_PATH.resolve()}")
print(f"📅 Using ISO Week:     {week_number} ({week_start_date} → {week_end_date})")
print(f"🧩 Filters provided:   {'yes' if raw_filters else 'no'}   → resolved: {filters}")
print(f"🔒 Locked snapshot:    {'yes' if any(EFFECTIVE_LOCKS.get(d) for d in DAYS_FULL) else 'no'}")
print("✅ Environment & parameters initialized.")


✅ Source data path:   C:\Users\Marco.Africani\OneDrive - AVU SA\AVU CPI Campaign\Puzzle_control_Reports\SOURCE_FILES
✅ Output data path:   C:\Users\Marco.Africani\OneDrive - AVU SA\AVU CPI Campaign\Puzzle_control_Reports\IRON_DATA
📦 Locked weeks path:  C:\Users\Marco.Africani\OneDrive - AVU SA\AVU CPI Campaign\Puzzle_control_Reports\IRON_DATA\locked_weeks
📄 Exports path:       C:\Users\Marco.Africani\OneDrive - AVU SA\AVU CPI Campaign\Puzzle_control_Reports\IRON_DATA\exports
📊 PowerBI path:       C:\Users\Marco.Africani\OneDrive - AVU SA\AVU CPI Campaign\Puzzle_control_Reports\IRON_DATA\powerbi
📅 Using ISO Week:     36 (2025-09-01 → 2025-09-07)
🧩 Filters provided:   yes   → resolved: {'loyalty': 'all', 'wine_type': None, 'bottle_size': 750, 'price_tier_bucket': '', 'last_stock': True, 'last_stock_threshold': 10, 'seasonality_boost': False, 'style': 'default', 'calendar_day': None}
🔒 Locked snapshot:    yes
✅ Environment & parameters initialized.


In [3]:
# --- CELL 2: HISTORY - Campaign History Ingestion (OMT main offer list) ---
# Builds history artifacts from OMT "main offer list" files and writes:
#   - OUTPUT_PATH/history/wine_campaign_history.json
#   - OUTPUT_PATH/campaign_index.json (by_id / by_name_vintage)
#   - OUTPUT_PATH/weekly_campaign_schedule_week_{WEEK}.json (flat arrays)
#   - OUTPUT_PATH/locked_weeks/locked_calendar_week_{WEEK}.json
#   - OUTPUT_PATH/weekly_leads_week_{WEEK}.json
#   - OUTPUT_PATH/schedule_index.json
from datetime import datetime, date, timedelta
import os, io, re, time, shutil, json
import pandas as pd
import numpy as np
from pathlib import Path
# --- Constants (shared with Cell 1) ---
NUM_SLOTS = 5
DAYS_FULL = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
DAYMAP = {1:"Monday",2:"Tuesday",3:"Wednesday",4:"Thursday",5:"Friday",6:"Saturday",7:"Sunday"}

# --- Paths (from Cell 1) ---
HISTORY_DIR = OUTPUT_PATH / "history"
HISTORY_DIR.mkdir(parents=True, exist_ok=True)
HISTORY_JSON = HISTORY_DIR / "wine_campaign_history.json"

LOCKED_PATH = OUTPUT_PATH / "locked_weeks"
LOCKED_PATH.mkdir(parents=True, exist_ok=True)

# Year/Week alignment with UI selection
CURRENT_YEAR = (week_start_date.year if "week_start_date" in globals() else datetime.now().year)

# -------------------------------
# 1) Discover OMT files (robust)
# -------------------------------
OMT_FILES = []
cands = list(SOURCE_PATH.glob("**/*.xlsx")) + list(SOURCE_PATH.glob("**/*.csv"))
for p in cands:
    # accept common variations: OMT ... (main)? ... (offer|campaign) ... list
    if re.search(r"omt.*(main)?.*(offer|campaign).*(list)", p.name, flags=re.I):
        OMT_FILES.append(p)
# Fallback: be a bit looser if nothing matched
if not OMT_FILES:
    for p in cands:
        if re.search(r"(offer|campaign).*(list)", p.name, flags=re.I):
            OMT_FILES.append(p)
OMT_FILES = sorted(set(OMT_FILES))
print(f"[HIST] Discovered {len(OMT_FILES)} OMT files:", [p.name for p in OMT_FILES] or "—")

# -----------------------------------------
# 2) Safe readers (work around file locking)
# -----------------------------------------
def _safe_read_xlsx(p: Path) -> pd.DataFrame:
    # Try read-bytes first (works even if file is open/locked sometimes)
    for attempt in range(3):
        try:
            with open(p, "rb") as f:
                buf = f.read()
            return pd.read_excel(io.BytesIO(buf), engine="openpyxl")
        except PermissionError as e:
            print(f"⏳ PermissionError on {p.name} (attempt {attempt+1}/3): {e}")
            time.sleep(0.7)
        except Exception as e:
            print(f"⚠️ Read-bytes failed on {p.name}: {e}")
            break  # if not permission, don't spin
    # Fallback: copy to TMP and read
    try:
        tmp = TMP_PATH / f"shadow_{int(time.time()*1000)}_{p.name}"
        shutil.copy2(p, tmp)
        return pd.read_excel(tmp, engine="openpyxl")
    except Exception as e:
        print(f"❌ Fallback copy+read failed for {p.name}: {e}")
        return pd.DataFrame()

def _safe_read_csv(p: Path) -> pd.DataFrame:
    for attempt in range(3):
        try:
            with open(p, "rb") as f:
                buf = f.read()
            return pd.read_csv(io.BytesIO(buf))
        except PermissionError as e:
            print(f"⏳ PermissionError on {p.name} (attempt {attempt+1}/3): {e}")
            time.sleep(0.7)
        except Exception as e:
            print(f"⚠️ Read-bytes failed on {p.name}: {e}")
            break
    try:
        tmp = TMP_PATH / f"shadow_{int(time.time()*1000)}_{p.name}"
        shutil.copy2(p, tmp)
        return pd.read_csv(tmp)
    except Exception as e:
        print(f"❌ Fallback copy+read failed for {p.name}: {e}")
        return pd.DataFrame()

def _load_omt_file(p: Path) -> pd.DataFrame:
    try:
        df = _safe_read_xlsx(p) if p.suffix.lower() == ".xlsx" else _safe_read_csv(p)
        if df.empty:
            print(f"⚠️ {p.name} loaded but EMPTY after safe read.")
        df.columns = [str(c).strip() for c in df.columns]
        return df
    except Exception as e:
        print(f"⚠️ Skipping {p.name}: {e}")
        return pd.DataFrame()

# --------------------------------
# 3) Column normalization & parse
# --------------------------------
ALIASES = {
    # datetime
    "Schedule DateTime":"schedule_dt","Scheduled":"schedule_dt","Scheduled At":"schedule_dt",
    "DateTime":"schedule_dt","schedule_datetime":"schedule_dt",
    "Date":"schedule_dt","Send Date":"schedule_dt","Offer Date":"schedule_dt",
    "Campaign Date":"schedule_dt","Scheduled Date":"schedule_dt","Scheduled On":"schedule_dt",
    "Sent At":"schedule_dt","Created":"schedule_dt","Execution Date":"schedule_dt",

    # ids
    "Wine ID":"id","wine_id":"id","ID":"id","Sku":"id","SKU":"id","SKU Code":"id",
    "Item Code":"id","Product ID":"id","Code":"id",

    # names
    "Wine":"wine","Name":"wine","Product Name":"wine","Product":"wine",
    "Product Title":"wine","Title":"wine","Label":"wine","Long Name":"wine","Display Name":"wine",

    # vintage
    "Vintage":"vintage","Year":"vintage",

    # type/colour
    "Full Type":"full_type","Type":"full_type","Colour":"full_type","Color":"full_type",

    # region
    "Region Group":"region_group","Region":"region_group","Origin":"region_group",

    # tier/price
    "Price Tier":"price_tier","Tier":"price_tier","Price":"price",
}

def _first(*vals, default=""):
    for v in vals:
        s = str(v).strip() if v is not None else ""
        if s:
            return s
    return default

def _canon_cols(df: pd.DataFrame) -> pd.DataFrame:
    orig = {c: df[c] for c in df.columns}

    # rename using aliases
    ren = {}
    for c in df.columns:
        if c in ALIASES:
            ren[c] = ALIASES[c]
    out = df.rename(columns=ren).copy()

    # schedule_dt: robust parse with EU formats; auto-detect if missing
    raw_dt = out.get("schedule_dt", None)
    parsed = None
    if raw_dt is not None:
        parsed = pd.to_datetime(raw_dt, errors="coerce", dayfirst=True, utc=False)

    if parsed is None or parsed.notna().sum() == 0:
        for c in out.columns:
            if re.search(r"(schedule|send|offer|campaign|date|time)", c, flags=re.I):
                try:
                    cand = pd.to_datetime(out[c], errors="coerce", dayfirst=True, utc=False)
                    if cand.notna().any():
                        parsed = cand
                        break
                except Exception:
                    pass
    out["schedule_dt"] = parsed if parsed is not None else pd.NaT

    # enforce strings
    for name, default in [("id",""),("wine",""),("vintage","NV"),
                          ("full_type",""),("region_group",""),("price_tier","")]:
        if name not in out.columns:
            out[name] = default
        out[name] = out[name].astype("string").fillna("").str.strip()

    # fix id like "123.0"
    out["id"] = out["id"].str.replace(r"\.0$", "", regex=True).replace({"nan": ""})
    # vintage blanks → NV
    out["vintage"] = out["vintage"].replace({"": "NV", "nan": "NV"})

    # backfill wine if blank from any plausible original column
    if out["wine"].eq("").any():
        candidates = ["Wine","Name","Product Name","Product","Product Title","Title","Label","Long Name","Display Name"]
        for c in candidates:
            if c in orig:
                mask = out["wine"].eq("") & orig[c].astype(str).fillna("").str.strip().ne("")
                out.loc[mask, "wine"] = orig[c].astype(str).str.strip()

    # derive price_tier from price if needed
    if "price" in out.columns and out["price_tier"].eq("").any():
        price_num = pd.to_numeric(out["price"], errors="coerce")
        def price_bucket(x):
            if pd.isna(x): return ""
            if x < 50:   return "Budget"
            if x < 100:  return "Mid-range"
            if x < 200:  return "Premium"
            if x < 500:  return "Luxury"
            return "Ultra Luxury"
        mask = out["price_tier"].eq("")
        out.loc[mask, "price_tier"] = price_num.map(price_bucket)

    # drop rows without a valid schedule_dt
    out = out[~out["schedule_dt"].isna()].copy()

    # ISO components
    iso = out["schedule_dt"].dt.isocalendar()
    out["iso_year"] = iso.year.astype(int)
    out["iso_week"] = iso.week.astype(int)
    out["iso_wday"] = iso.day.astype(int)
    return out

# -------------------------------------
# 4) Load files & combine into history
# -------------------------------------
frames = []
for p in OMT_FILES:
    df = _load_omt_file(p)
    if not df.empty:
        frames.append(_canon_cols(df))

history_df = (
    pd.concat(frames, ignore_index=True)
    if frames else
    pd.DataFrame(columns=["schedule_dt","id","wine","vintage","full_type","region_group","price_tier","iso_year","iso_week","iso_wday"])
)

if history_df.empty:
    print("ℹ️ No OMT history files discovered or parsable; skipping history build.")
    OMT_HISTORY_MAP = {}
else:
    # 5) Build last-campaign map (by id, with fallback to wine::vintage)
    # max schedule_dt per id
    last_seen_by_id = (
        history_df.sort_values("schedule_dt")
        .groupby("id", dropna=False)["schedule_dt"]
        .max()
    )

    # fallback key if id is blank
    history_df["name_key"] = history_df["wine"].str.strip() + "::" + history_df["vintage"].str.strip()
    last_seen_by_name = (
        history_df.sort_values("schedule_dt")
        .groupby("name_key")["schedule_dt"]
        .max()
    )

    # Compose history map keyed by id if present else name_key
    OMT_HISTORY_MAP = {}
    for _, row in history_df.iterrows():
        key = (row["id"].strip() or row["name_key"])
        if key not in OMT_HISTORY_MAP:
            OMT_HISTORY_MAP[key] = {"wine": row["wine"], "vintage": row["vintage"], "dates": []}

    # attach last_campaign_date
    for k in list(OMT_HISTORY_MAP.keys()):
        if k in last_seen_by_id.index and pd.notna(last_seen_by_id.loc[k]):
            dt = last_seen_by_id.loc[k]
        else:
            dt = last_seen_by_name.loc[k] if k in last_seen_by_name.index else None
        try:
            if pd.notna(dt):
                OMT_HISTORY_MAP[k]["last_campaign_date"] = pd.to_datetime(dt).date().isoformat()
        except Exception:
            pass

    # persist history map
    HISTORY_JSON.write_text(json.dumps(OMT_HISTORY_MAP, indent=2), encoding="utf-8")
    print(f"✅ History map written → {HISTORY_JSON.name} (keys: {len(OMT_HISTORY_MAP)})")

    # 6) campaign_index.json for fast merges (exporter cell consumes this)
    campaign_index = {"by_id": {}, "by_name_vintage": {}}
    for k, rec in OMT_HISTORY_MAP.items():
        last_dt = rec.get("last_campaign_date")
        if not last_dt:
            continue
        if "::" in k:
            campaign_index["by_name_vintage"][k] = {"last_campaign_date": last_dt}
        else:
            campaign_index["by_id"][k] = {"last_campaign_date": last_dt}
    (OUTPUT_PATH / "campaign_index.json").write_text(json.dumps(campaign_index, indent=2), encoding="utf-8")
    print("🧭 campaign_index.json written (by_id / by_name_vintage).")

    # 7) Build week calendars across ALL years in the history (so previous weeks populate correctly)
    def _build_week_calendar(sub_df: pd.DataFrame) -> dict:
        cal = {d: [] for d in DAYS_FULL}
        seen_keys = set()
        for r in sub_df.sort_values("schedule_dt").to_dict("records"):
            dname = DAYMAP.get(int(r.get("iso_wday", 1)), "Monday")
            wine_key = (str(r.get("id") or "").strip()
                        or f"{(r.get('wine') or '').strip()}::{(r.get('vintage') or 'NV').strip()}")
            item = {
                "id": str(r.get("id","") or ""),
                "wine": r.get("wine") or "Unknown",
                "vintage": (r.get("vintage") or "NV"),
                "full_type": r.get("full_type") or "Unknown",
                "region_group": r.get("region_group") or "Unknown",
                "price_tier": r.get("price_tier") or "",
                "stock": None,
                "match_quality": "History (Locked)",
                "avg_cpi_score": 0,
                "locked": True,
                "last_campaign_date": pd.to_datetime(r["schedule_dt"]).date().isoformat(),
            }
            if len(cal[dname]) < NUM_SLOTS and wine_key not in seen_keys:
                cal[dname].append(item); seen_keys.add(wine_key)
        return cal
    
    # Build nested {year: {week: calendar}}
    weeks_by_year = {}
    for (yr, wk), sub in history_df.groupby(["iso_year", "iso_week"]):
        weeks_by_year.setdefault(int(yr), {})[int(wk)] = _build_week_calendar(sub)
    
    # 8) Backfill window: ensure we have files for [current week - BACKFILL_WEEKS ... current week]
    BACKFILL_WEEKS = int(os.getenv("BACKFILL_WEEKS", "1"))  # includes the previous week by default
    def _iso_prev(yr, wk):
        # previous ISO week (handling year boundary)
        try:
            d = date.fromisocalendar(yr, wk, 1) - timedelta(days=7)
            iso = d.isocalendar()
            return iso.year, iso.week
        except Exception:
            return yr, max(1, wk-1)
    
    need_pairs = set()
    cy, cw = calendar_year, week_number
    y, w = cy, cw
    for i in range(BACKFILL_WEEKS + 1):  # +1 to include current week
        need_pairs.add((y, w))
        y, w = _iso_prev(y, w)
    
    # 9) Persist per-week artifacts + index (year-aware + legacy)
    def _atomic_write(path: Path, text: str):
        tmp = path.with_suffix(path.suffix + ".tmp")
        tmp.write_text(text, encoding="utf-8")
        os.replace(tmp, path)
    
    index_path = OUTPUT_PATH / "schedule_index.json"
    try:
        idx = json.loads(index_path.read_text(encoding="utf-8")) if index_path.exists() else {}
    except Exception:
        idx = {}
    
    latest_year = None
    latest_week = None
    
    for yr, wkmap in weeks_by_year.items():
        for wk, cal_flat in wkmap.items():
            # If you want to persist only the backfill window, uncomment next line:
            # if (yr, wk) not in need_pairs: continue
    
            # a) Year+week primary
            out_json_year = OUTPUT_PATH / f"weekly_campaign_schedule_{yr}_week_{wk}.json"
            _atomic_write(out_json_year, json.dumps(cal_flat, indent=2))
    
            # b) Legacy (same content) for compatibility
            out_json_legacy = OUTPUT_PATH / f"weekly_campaign_schedule_week_{wk}.json"
            _atomic_write(out_json_legacy, json.dumps(cal_flat, indent=2))
    
            # c) Locked snapshot (arrays of length NUM_SLOTS)
            locked_week = {
                day: [(cal_flat[day][i] if i < len(cal_flat[day]) else None) for i in range(NUM_SLOTS)]
                for day in DAYS_FULL
            }
            out_locked_year = LOCKED_PATH / f"locked_calendar_{yr}_week_{wk}.json"
            _atomic_write(out_locked_year, json.dumps(locked_week, indent=2))
    
            # d) Leads (empty buckets)
            leads_json = OUTPUT_PATH / f"weekly_leads_{yr}_week_{wk}.json"
            _atomic_write(leads_json, json.dumps({"Tuesday-Thursday": [], "Thursday-Friday": []}, indent=2))
    
            # e) Index (nested by year→week)
            idx.setdefault(str(yr), {})[str(wk)] = {
                "json": out_json_year.name,
                "updated_at": datetime.now().isoformat(timespec="seconds")
            }
            if (latest_year is None) or (yr > latest_year) or (yr == latest_year and wk > (latest_week or 0)):
                latest_year, latest_week = yr, wk
    
    # Convenience pointers
    if latest_year is not None and latest_week is not None:
        idx["_latest_year"] = str(latest_year)
        idx["_latest_week"] = str(latest_week)
    
    _atomic_write(index_path, json.dumps(idx, indent=2))
    print(f"📅 Wrote {sum(len(w) for w in weeks_by_year.values())} week files across {len(weeks_by_year)} year(s).")
    print(f"🔒 Locked calendars → {LOCKED_PATH.name}")
    print("🧹 Leads files created empty so leads boxes start blank each session.")



[HIST] Discovered 1 OMT files: ['OMT Main Offer List.xlsx']


✅ History map written → wine_campaign_history.json (keys: 53)
🧭 campaign_index.json written (by_id / by_name_vintage).


📅 Wrote 129 week files across 4 year(s).
🔒 Locked calendars → locked_weeks
🧹 Leads files created empty so leads boxes start blank each session.


In [4]:
# --- CELL 3: Set Occasion/Subfolder Path ---
# Creates a deterministic subfolder for this run inside REPORTS_PATH.
# Safe with/without papermill, and robust if OCCASION is empty.

from datetime import datetime, timedelta, date
from pathlib import Path
import os
import re

# Fallback for _iso_week_now_europe if Cell 1 wasn't executed
try:
    _iso_week_now_europe  # type: ignore
except NameError:
    try:
        from zoneinfo import ZoneInfo
    except Exception:
        ZoneInfo = None  # type: ignore
    def _iso_week_now_europe():
        now = datetime.now(ZoneInfo("Europe/Zurich")) if ZoneInfo else datetime.now()
        iso = now.isocalendar()
        return iso.year, iso.week

# Use selected week/year from Cell 1 when available
try:
    _wk = int(week_number)
except Exception:
    _wk = _iso_week_now_europe()[1]

try:
    _year = int(calendar_year)
except Exception:
    _year = _iso_week_now_europe()[0]

def _week_bounds(iso_week: int, year: int):
    try:
        start = date.fromisocalendar(year, iso_week, 1)
    except ValueError:
        start = date.fromisocalendar(year, min(max(iso_week, 1), 52), 1)
    end = start + timedelta(days=6)
    return start, end

# Prefer globals from Cell 1 if present
try:
    _week_start = week_start_date; _week_end = week_end_date
except Exception:
    _week_start, _week_end = _week_bounds(_wk, _year)

def _season_from_week(week: int) -> str:
    if 9 <= week <= 21:
        return "Spring"
    elif 22 <= week <= 35:
        return "Summer"
    elif 36 <= week <= 48:
        return "Autumn"
    else:
        return "Winter"

_season = _season_from_week(_wk)

# --- Read OCCASION from globals/papermill/env; keep your default ---
try:
    OCCASION  # type: ignore  # if already defined above, leave it
except NameError:
    OCCASION = os.getenv("OCCASION", "Notreceived")

# Optional “forced_day” kept for compatibility; ignored if falsy
try:
    forced_day  # type: ignore
except NameError:
    forced_day = None

def _slugify(s: str) -> str:
    s = (s or "").strip()
    if not s:
        return ""
    s = s.replace("&", "and").replace("/", "-")
    s = re.sub(r"[^A-Za-z0-9_-]+", "_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s or ""

# Compose a stable, human-readable folder name
_occ_slug = _slugify(str(OCCASION))
_base = f"{_year}_W{_wk:02d}_{_season}"
folder_name = f"{_base}_{_occ_slug}" if _occ_slug else (_slugify(str(forced_day)) or _base)

# Ensure REPORTS_PATH exists (from Cell 1); fall back to IRON_DATA/non_recipient_reports if missing
try:
    REPORTS_PATH
except NameError:
    REPORTS_PATH = Path.home() / "OneDrive - AVU SA" / "AVU CPI Campaign" / "non_recipient_reports"
REPORTS_PATH.mkdir(parents=True, exist_ok=True)

report_subfolder = REPORTS_PATH / folder_name
report_subfolder.mkdir(parents=True, exist_ok=True)

# Also expose a canonical var some downstream cells may prefer
REPORT_RUN_PATH = report_subfolder

print(
    f"📅 Week {_wk} "
    f"({_week_start.strftime('%b %d')} – {_week_end.strftime('%b %d')}) "
    f"| Season: {_season}"
)
print(f"🗂  Occasion: {OCCASION!r}  → Folder: {folder_name}")
print(f"✅ Report subfolder ready at: {report_subfolder}")


📅 Week 36 (Sep 01 – Sep 07) | Season: Autumn
🗂  Occasion: 'Notreceived'  → Folder: 2025_W36_Autumn_Notreceived
✅ Report subfolder ready at: C:\Users\Marco.Africani\OneDrive - AVU SA\AVU CPI Campaign\non_recipient_reports\2025_W36_Autumn_Notreceived


In [5]:
# --- CELL 4: Load & Filter Clients for Targeting (Cloud-Safe) ---
# Loads Lines.xlsx and Power BI stats, cleans & filters, and writes a compact
# pickle for downstream cells. Robust to Excel float IDs (e.g., "12345.0").

from pathlib import Path
import pandas as pd
import numpy as np

# ---------- Inputs / Outputs ----------
lines_path = SOURCE_PATH / "Lines.xlsx"
stats_path = SOURCE_PATH / "Power BI Dtld. Statistics ALL.xlsx"
output_temp_path = OUTPUT_PATH / "filtered_clients.pkl"

def _ensure_file(p: Path, label: str):
    if not p.exists() or p.stat().st_size == 0:
        print(f"❌ Missing or empty {label}: {p}")
        return False
    return True

def _norm_id_series(s: pd.Series) -> pd.Series:
    """Normalize Excel-ish ID columns:
       - cast to string, strip
       - drop trailing '.0'
       - collapse common null tokens to NaN
    """
    if s is None:
        return pd.Series(dtype="string")
    out = (
        s.astype(str).str.strip()
         .str.replace(r"\.0$", "", regex=True)
         .replace({r"^(nan|na|null|none|\s*)$", np.nan}, regex=True, inplace=False)
    )
    return out

# ---------- Load ----------
ok_lines = _ensure_file(lines_path, "Lines.xlsx")
ok_stats = _ensure_file(stats_path, "Power BI Stats.xlsx")

if not (ok_lines and ok_stats):
    # Write empty output so downstream cells can still run gracefully
    empty = pd.DataFrame(columns=["customer_no","spanish_customer_no","total_bottle_amount_lcy"])
    empty.to_pickle(output_temp_path)
    clients_df = empty.copy()
    print("⚠️ One or more inputs missing. Wrote empty filtered_clients.pkl and continuing.")
else:
    # Read Excel (engine chosen by pandas)
    lines_df = pd.read_excel(lines_path)
    stats_df = pd.read_excel(stats_path)

    # Clean column names
    lines_df.columns = lines_df.columns.str.strip()
    stats_df.columns = stats_df.columns.str.strip()

    print("📋 Columns in Lines:", lines_df.columns.tolist())
    print("📋 Columns in Stats:", stats_df.columns.tolist())

    # ---------- Validate & Filter (Excluded) ----------
    if "Excluded" not in lines_df.columns:
        raise KeyError("❌ 'Excluded' column not found in Lines.xlsx.")
    
    excluded_norm = lines_df["Excluded"].apply(lambda v: str(v).strip().lower() if pd.notna(v) else "")
    # Treat only explicit truthy values as excluded
    truthy_exclude = {"true", "1", "yes", "y", "t"}
    keep_mask = ~excluded_norm.isin(truthy_exclude)
    filtered_lines = lines_df.loc[keep_mask].copy()
    print(f"🧮 Filtered lines count: {len(filtered_lines)}")
    if filtered_lines.empty:
        print("⚠️ No lines passed the Excluded=False filter.")

    # ---------- Extract and normalize contact numbers ----------
    contact_col = None
    for c in ["Contact No.", "Contact No", "Customer No.", "Customer No", "ContactNo", "CustomerNumber"]:
        if c in filtered_lines.columns:
            contact_col = c
            break
    if contact_col is None:
        raise KeyError("❌ 'Contact No.' column not found in Lines.xlsx.")

    contact_nos = _norm_id_series(filtered_lines[contact_col]).dropna().unique().tolist()
    print("🔍 Sample Contact Nos:", contact_nos[:5])

    # ---------- Validate required stats columns ----------
    required_cols = ["Customer No.", "Spanish Customer No.", "Total Bottle Amount (LCY)"]
    missing = [c for c in required_cols if c not in stats_df.columns]
    if missing:
        raise KeyError(f"❌ Missing columns in Power BI Stats file: {missing}")

    # Normalize IDs and amounts
    stats_df["Customer No."] = _norm_id_series(stats_df["Customer No."])
    stats_df["Spanish Customer No."] = _norm_id_series(stats_df["Spanish Customer No."])
    stats_df["Total Bottle Amount (LCY)"] = pd.to_numeric(
        stats_df["Total Bottle Amount (LCY)"], errors="coerce"
    ).fillna(0.0)

    # ---------- Business rule for client 122636 (vectorized) ----------
    # If Customer No. == '122636' AND Spanish Customer No. is one of contact_nos:
    #   - overwrite Customer No. with Spanish Customer No.
    #   - multiply Total Bottle Amount (LCY) by 1.12
    mask_122636 = (stats_df["Customer No."] == "122636") & (
        stats_df["Spanish Customer No."].isin(contact_nos)
    )
    if mask_122636.any():
        stats_df.loc[mask_122636, "Customer No."] = stats_df.loc[mask_122636, "Spanish Customer No."]
        stats_df.loc[mask_122636, "Total Bottle Amount (LCY)"] *= 1.12
        print(f"🔧 Applied 1.12x adjustment to {mask_122636.sum()} rows for client 122636.")

    # ---------- Final filter: keep only valid clients ----------
    final_df = stats_df.loc[
        stats_df["Customer No."].isin(contact_nos)
    ].copy()

    # ---------- Normalize output schema ----------
    final_df = final_df.rename(
        columns={
            "Customer No.": "customer_no",
            "Spanish Customer No.": "spanish_customer_no",
            "Total Bottle Amount (LCY)": "total_bottle_amount_lcy",
        }
    )
    # keep only the essentials + anything you know downstream needs
    core_cols = ["customer_no", "spanish_customer_no", "total_bottle_amount_lcy"]
    # If there are extra useful columns, keep them but put core first
    extras = [c for c in final_df.columns if c not in core_cols]
    final_df = final_df[core_cols + extras]

    # Persist compact binary for downstream cells
    final_df.to_pickle(output_temp_path)

    # Expose for the rest of the notebook
    clients_df = final_df

    # ---------- Logs ----------
    print(f"✅ Final shape: {final_df.shape}")
    print(f"📁 Internal .pkl saved to: {output_temp_path}")
    if final_df.empty:
        print("⚠️ The output file contains only headers — no matching customers found.")
    else:
        print("👀 Sample rows:")
        try:
            from IPython.display import display  # safe if IPython present
            display(final_df.head(3))
        except Exception:
            print(final_df.head(3).to_string(index=False))


📋 Columns in Lines: ['Salesperson Code', 'Total Order No', 'Last Offer Sent Date', 'Turnover (€)', 'Sent Emails Today', 'Sent Emails Scheduled', 'Excluded', 'Unsubscribed', 'Contact Company Name', 'Contact Person Name', 'Country', 'City', 'Language Code', 'Contact Email', 'Contact Email CC', 'Email Sent', 'Send Email Error', 'Contact Phone No.', 'Contact Mobile Phone No.', 'Informal', 'Contact No.', 'Included as New Customer', 'First Purchase Date', 'Bought Wine with exceed price', 'Bought Wine in Parent Campaign(Recall)', 'Unsubsc. for Recall', 'Buy from Competitor', 'Cont. Offer Currency']
📋 Columns in Stats: ['Period', 'Country Code', 'Customer Post Code', 'Customer City', 'Customer Net Worth', 'Spanish Customer No.', 'Spanish Customer Name', 'Wine Country', 'Wine Overall Type', 'Wine Region', 'Wine Sub-Region', 'Wine Type', 'Classification', 'Color Code', 'Producer No.', 'Wine No.', 'Wine Name', 'Producer Name', 'Item Sales Price from Price List', 'Item No.', 'Customer No.', 'Custo

🔧 Applied 1.12x adjustment to 9 rows for client 122636.
✅ Final shape: (1005, 52)
📁 Internal .pkl saved to: C:\Users\Marco.Africani\OneDrive - AVU SA\AVU CPI Campaign\Puzzle_control_Reports\IRON_DATA\filtered_clients.pkl
👀 Sample rows:


,customer_no,spanish_customer_no,total_bottle_amount_lcy,Period,Country Code,Customer Post Code,Customer City,Customer Net Worth,Spanish Customer Name,Wine Country,...,Denomination Code,Item Category Code,Item Real Stock,Item Stock,Campaign No.,Document Currency Factor,Document Canceled,PA,PV Total,Salesperson
1337,101332,nan,1108.912355,2025,LI,9494,Schaan,0,NaN,IT,...,NaN,WINE,0,0,CM-25-01069,0,False,0,0,N.BOLDRINI
1374,101332,nan,1448.834413,2025,LI,9494,Schaan,0,NaN,US,...,NaN,WINE,0,0,CM-25-00459,0,False,0,0,N.BOLDRINI
1375,101332,nan,2724.413939,2025,LI,9494,Schaan,0,NaN,FR,...,NaN,WINE,0,0,NaN,0,False,0,0,N.BOLDRINI


In [6]:
# --- CELL 5: Enhance Stock Dataset (Detailed Stock + OMT Info) ---
# Robust load, normalize, enrich with price tiers (label + stable key),
# trait inference, and OMT campaign summary. Outputs stock_df_final.pkl.

from pathlib import Path
import pandas as pd
import numpy as np
import re

# --- Safe unidecode import (fallback) ---
try:
    from unidecode import unidecode  # pip install Unidecode
except Exception:
    import unicodedata
    def unidecode(x):
        x = '' if x is None else str(x)
        return ''.join(
            ch for ch in unicodedata.normalize('NFKD', x)
            if not unicodedata.combining(ch)
        )

stock_path = SOURCE_PATH / "Detailed Stock List.xlsx"
omt_path   = SOURCE_PATH / "OMT Main Offer List.xlsx"

def _ensure_file(p: Path, label: str) -> bool:
    if not p.exists() or p.stat().st_size == 0:
        print(f"❌ Missing or empty {label}: {p}")
        return False
    return True

ok_stock = _ensure_file(stock_path, "Detailed Stock List.xlsx")
ok_omt   = _ensure_file(omt_path, "OMT Main Offer List.xlsx")

if not ok_stock:
    # Minimal empty frame so downstream cells can still run.
    stock_df = pd.DataFrame(columns=[
        "id","wine","producer","classification","region","type","color","size","vintage",
        "origin","stock","CHF Price","price_tier","price_tier_key","full_type","region_group",
        "bottle_size_ml","grape_list","body","sweetness","occasion","wine_full_match_string",
        "Num_of_CM","most_recent_date","last_eur_price","last_chf_price","number_of_sent_emails"
    ])
    final_stock_path = OUTPUT_PATH / "stock_df_final.pkl"
    stock_df.to_pickle(final_stock_path)
    print(f"⚠️ No stock file. Wrote empty dataset to: {final_stock_path}")
else:
    # ---- Load Detailed Stock (header fallback: row 2 first, then row 0) ----
    try:
        detailed_df = pd.read_excel(stock_path, header=2)
        # If the expected key columns aren't present, retry with header=0
        if not any(k in detailed_df.columns for k in ["ID","Item No.","Wine","Stock"]):
            detailed_df = pd.read_excel(stock_path, header=0)
    except Exception:
        detailed_df = pd.read_excel(stock_path, header=0)

    # ---- Flexible rename map (accept common variants) ----
    print("📋 Stock columns (raw):", detailed_df.columns.tolist())
    rename_map_variants = {
        "ID": "id",
        "Item No.": "id",
        "Wine": "wine",
        "Producer": "producer",
        "Classification": "classification",
        "Region": "region",
        "Type": "type",
        "Color": "color",
        "Size": "size",
        "Vintage": "vintage",
        "Origin": "origin",
        "Stock": "stock",
        "Qty": "stock",
        "CHF p/bt VAT excl.": "CHF Price",
        "CHF p/bt (VAT excl.)": "CHF Price",
        "Unit Price (CHF)": "CHF Price",
        "CHF Price": "CHF Price",
        "AVG": "avg_score",
        "Average Score": "avg_score",
    }
    actual_map = {c: rename_map_variants[c] for c in detailed_df.columns if c in rename_map_variants}
    detailed_df = detailed_df.rename(columns=actual_map)

    # ---- Ensure core columns exist ----
    for c in ["id","wine","producer","classification","region","type","color","size","vintage","origin","stock","CHF Price","avg_score"]:
        if c not in detailed_df.columns:
            detailed_df[c] = np.nan

    # ---- Normalize critical fields ----
    detailed_df["id"] = (
        detailed_df["id"].astype(str).str.strip()
        .str.replace(r"\.0$", "", regex=True)
        .replace({"nan": "", "None": ""})
    )
    # CHF Price → numeric
    detailed_df["CHF Price"] = (
        detailed_df["CHF Price"]
        .astype(str).str.replace(r"[^\d\.,-]", "", regex=True)
        .str.replace(",", ".", regex=False)
    )
    detailed_df["CHF Price"] = pd.to_numeric(detailed_df["CHF Price"], errors="coerce")
    # Stock → numeric, missing → 0
    detailed_df["stock"] = pd.to_numeric(detailed_df["stock"], errors="coerce").fillna(0)

    # ---- OMT Campaign Summary (robust headers & normalization) ----
    if not ok_omt:
        campaign_summary = pd.DataFrame(columns=[
            "Item No.","Num_of_CM","most_recent_date","last_eur_price","last_chf_price","number_of_sent_emails"
        ])
    else:
        omt_df = pd.read_excel(omt_path)

        # Canonicalize common OMT headers
        id_aliases    = ["Item No.","Item No","Item","ID","Wine ID","Sku","SKU","SKU Code","Item Code","Product ID","Code"]
        sched_aliases = ["Schedule DateTime","Scheduled","Scheduled At","DateTime","Date","Send Date","Offer Date",
                         "Campaign Date","Scheduled Date","Scheduled On","Sent At","Created","Execution Date"]
        eur_price_aliases = ["Unit Price (EUR)","EUR Unit Price","EUR Price","Price (EUR)"]
        chf_price_aliases = ["Unit Price (CHF)","CHF Unit Price","CHF Price","Unit Price"]
        sent_aliases  = ["Number of Sent Emails","Sent Emails","Emails Sent"]

        def _pick(cands, df):
            for c in cands:
                if c in df.columns: return c
            return None

        id_col    = _pick(id_aliases, omt_df)
        # If no ID, we can't summarize meaningfully
        if id_col is None:
            campaign_summary = pd.DataFrame(columns=[
                "Item No.","Num_of_CM","most_recent_date","last_eur_price","last_chf_price","number_of_sent_emails"
            ])
        else:
            sched_col = _pick(sched_aliases, omt_df)
            eur_col   = _pick(eur_price_aliases, omt_df)
            chf_col   = _pick(chf_price_aliases, omt_df)
            sent_col  = _pick(sent_aliases, omt_df)

            omt_df = omt_df.rename(columns={id_col: "Item No."})
            omt_df["Item No."] = (
                omt_df["Item No."].astype(str).str.strip()
                .str.replace(r"\.0$", "", regex=True)
            )

            if sched_col:
                omt_df["Schedule DateTime"] = pd.to_datetime(omt_df[sched_col], errors="coerce", dayfirst=True, utc=False)
            else:
                omt_df["Schedule DateTime"] = pd.NaT

            omt_df["Unit Price (EUR)"] = pd.to_numeric(omt_df[eur_col], errors="coerce") if eur_col else np.nan
            omt_df["Unit Price"]       = pd.to_numeric(omt_df[chf_col], errors="coerce") if chf_col else np.nan
            omt_df["Number of Sent Emails"] = pd.to_numeric(omt_df[sent_col], errors="coerce") if sent_col else np.nan

            # Sort so “last” really is most recent in each group
            omt_df = omt_df.sort_values(["Item No.","Schedule DateTime"], ascending=[True, True])

            counts = omt_df.groupby("Item No.").size().rename("Num_of_CM")
            last_rows = omt_df.groupby("Item No.").tail(1)[
                ["Item No.","Schedule DateTime","Unit Price (EUR)","Unit Price","Number of Sent Emails"]
            ]

            campaign_summary = (
                last_rows.rename(columns={
                    "Schedule DateTime": "most_recent_date",
                    "Unit Price (EUR)": "last_eur_price",
                    "Unit Price": "last_chf_price",
                    "Number of Sent Emails": "number_of_sent_emails",
                })
                .merge(counts, left_on="Item No.", right_index=True, how="left")
            )

    # ---- Price tier logic (label + stable key) ----
    def price_tier_label(price):
        try:
            p = float(price)
            if p < 50:   return "Budget"
            if p < 100:  return "Mid-range"
            if p < 200:  return "Premium"
            if p < 500:  return "Luxury"
            return "Ultra Luxury"
        except Exception:
            return None

    def price_tier_key_from_label(label):
        m = (label or "").strip().lower()
        return {
            "budget": "budget",
            "mid-range": "mid",
            "premium": "premium",
            "luxury": "luxury",
            "ultra luxury": "ultra",
        }.get(m, "")

    detailed_df["price_tier"] = detailed_df["CHF Price"].apply(price_tier_label)
    detailed_df["price_tier_key"] = detailed_df["price_tier"].apply(price_tier_key_from_label)

    # ---- Merge OMT summary → stock ----
    enhanced_df = detailed_df.merge(
        campaign_summary, how="left", left_on="id", right_on="Item No."
    )

    # Region group fallback (handle empty strings)
    enhanced_df["region_group"] = (
        enhanced_df["origin"].replace("", np.nan)
        .fillna(enhanced_df["region"])
    )
    enhanced_df["most_recent_date"] = pd.to_datetime(enhanced_df["most_recent_date"], errors="coerce")

    # ---- Helpers / inference ----
    def infer_type_class(row):
        t = unidecode(str(row.get("type", "")).lower())
        c = unidecode(str(row.get("color", "")).lower())
        txt = f"{t} {c}"
        if "sparkling" in txt or "champagne" in txt or "cava" in txt or "prosecco" in txt:
            return "Sparkling"
        if "dessert" in txt or "sweet" in txt or "sauternes" in txt or "porto" in txt:
            return "Dessert"
        if "rose" in txt or "rosé" in txt:
            return "Rose"
        if "white" in txt or "blanc" in txt or "bianco" in txt:
            return "White"
        if "red" in txt or "rouge" in txt or "rosso" in txt:
            return "Red"
        # Default to wine "type" signal if available
        return "Red" if "red" in t else ("White" if "white" in t else "Red")

    def parse_bottle_size_ml(val):
        s = unidecode(str(val)).lower().strip()
        if not s or s == "nan": return np.nan
        # named formats
        if "jeroboam" in s: return 3000
        if "magnum" in s:   return 1500
        # normalize units
        s = s.replace("lt", "l")
        # litres forms like "0.75 l"
        if "l" in s and "ml" not in s and "cl" not in s:
            num = re.sub(r"[^\d\.]", "", s)
            try: return int(float(num) * 1000)  # L → ml
            except Exception: return np.nan
        # numeric like "75cl" or "750ml"
        s = s.replace("cl","").replace("ml","").strip()
        s = re.sub(r"[^\d\.]", "", s)
        try:
            num = float(s)
            # if < 100, assume cl → convert to ml
            return int(num * 10) if num < 100 else int(num)
        except Exception:
            return np.nan

    def infer_grapes(row):
        wine_name = unidecode(str(row.get('wine', '')).lower().strip())
        producer  = unidecode(str(row.get('producer', '')).lower().strip())
        origin    = unidecode(str(row.get('origin', '')).lower().strip())
        combined  = f"{wine_name} {producer} {origin}"

        grape_keywords = {
            'nebbiolo':'Nebbiolo','tempranillo':'Tempranillo','cabernet sauvignon':'Cabernet Sauvignon',
            ' cabernet ':'Cabernet Sauvignon',' cab ':'Cabernet Sauvignon','merlot':'Merlot',
            'pinot noir':'Pinot Noir',' pinot ':'Pinot Noir','sangiovese':'Sangiovese','syrah':'Syrah',
            'shiraz':'Syrah','grenache':'Grenache','chardonnay':'Chardonnay',' chard ':'Chardonnay',
            'riesling':'Riesling','sauvignon blanc':'Sauvignon Blanc',' sauvignon ':'Sauvignon Blanc',
            ' sauv ':'Sauvignon Blanc','zinfandel':'Zinfandel','primitivo':'Primitivo','malbec':'Malbec',
            'grigio':'Pinot Grigio','garganega':'Garganega',"nero d avola":"Nero d'Avola",'barbera':'Barbera',
            'carmenere':'Carmenère','trebbiano':'Trebbiano','vermentino':'Vermentino','teroldego':'Teroldego'
        }
        hits = {g for k,g in grape_keywords.items() if k in f" {combined} "}
        if not hits:
            t = unidecode(str(row.get('type',''))).lower()
            if 'red' in t: hits.add('Red Blend')
            elif 'white' in t: hits.add('White Blend')
            elif 'sparkling' in t: hits.add('Sparkling Blend')
            elif 'rose' in t or 'rosé' in t: hits.add('Rosé Blend')
            else: hits.add('Unknown')
        return '/'.join(sorted(hits))

    def infer_body(row):
        c = str(row.get("color","")).lower()
        t = str(row.get("type","")).lower()
        if "sparkling" in t: return 2
        if "red" in c: return 4
        if "white" in c or "ros" in c: return 2
        return 3

    def infer_sweetness(row):
        t = str(row.get("type","")).lower()
        if "brut" in t or "dry" in t: return 1
        if "sweet" in t: return 5
        if "medium" in t: return 3
        return 3

    def infer_occasion(row):
        tier = row.get("price_tier")
        t = str(row.get("type","")).lower()
        if tier in ["Luxury","Ultra Luxury"]: return "Gifting"
        if "sparkling" in t or "ros" in t:     return "Celebration"
        if tier in ["Mid-range","Premium"]:    return "Dinner"
        return "Casual"

    # Ensure text cols exist to avoid "nan" literals
    for tcol in ["wine","producer","region","type","color","size","vintage"]:
        enhanced_df[tcol] = enhanced_df[tcol].fillna("").astype(str)

    enhanced_df["type_class"]     = enhanced_df.apply(infer_type_class, axis=1)
    enhanced_df["bottle_size_ml"] = enhanced_df["size"].apply(parse_bottle_size_ml)
    enhanced_df["grape_list"]     = enhanced_df.apply(infer_grapes, axis=1)
    enhanced_df["body"]           = enhanced_df.apply(infer_body, axis=1)
    enhanced_df["sweetness"]      = enhanced_df.apply(infer_sweetness, axis=1)
    enhanced_df["occasion"]       = enhanced_df.apply(infer_occasion, axis=1)

    # --- full_type: vectorized ---
    t = enhanced_df["type"].fillna("").astype(str).str.strip()
    c = enhanced_df["color"].fillna("").astype(str).str.strip()
    enhanced_df["full_type"] = (t.str.title() + (" " + c.str.title()).where(c.ne(""), "")).str.strip()

    # Stable dedupe key for scheduling logic if needed
    enhanced_df["wine_full_match_string"] = (
        enhanced_df["wine"].str.strip() + " " +
        enhanced_df["producer"].str.strip() + " " +
        enhanced_df["vintage"].astype(str).str.strip() + " " +
        enhanced_df["size"].astype(str).str.strip() + " " +
        enhanced_df["id"].astype(str).str.strip()
    ).str.replace(r"\s+", " ", regex=True).str.strip()

    # Final save
    final_stock_path = OUTPUT_PATH / "stock_df_final.pkl"
    enhanced_df.to_pickle(final_stock_path)

    stock_df = enhanced_df  # expose to later cells

    print(f"✅ Final wine dataset enriched and saved to: {final_stock_path}")
    print(f"   Rows: {len(enhanced_df):,} | With price tiers: {enhanced_df['price_tier'].notna().sum():,}")
    try:
        from IPython.display import display
        display(enhanced_df.head(3))
    except Exception:
        print(enhanced_df.head(3).to_string(index=False))


📋 Stock columns (raw): ['ID', 'Stock', 'Minimum Order Q-ty', 'Size', 'Wine', 'Producer', 'Origin', 'Classification', 'Region', 'Country', 'Color', 'Type', 'Vintage', 'Rating', 'RP', 'WA', 'WS', 'JS', 'RG', 'VINOUS', 'JR', 'Decanter', 'NM', 'JA', 'AG', 'Falstaff', 'JD', 'DB', 'WInd', 'WCI', 'YB', 'AMA', 'JMQ', 'MDM', 'VINUM', 'AVG', 'EUR p/bt', 'CHF p/bt VAT excl.', 'CHF p/bt VAT incl.', 'OMT Last Offer Price CHF', 'OMT Last Offer Price Euro', 'OMT last offer date']


✅ Final wine dataset enriched and saved to: C:\Users\Marco.Africani\OneDrive - AVU SA\AVU CPI Campaign\Puzzle_control_Reports\IRON_DATA\stock_df_final.pkl
   Rows: 4,447 | With price tiers: 4,447


,id,stock,Minimum Order Q-ty,size,wine,producer,origin,classification,region,Country,...,Num_of_CM,region_group,type_class,bottle_size_ml,grape_list,body,sweetness,occasion,full_type,wine_full_match_string
0,40261,108,12,75.0,Bolaire,Château Bolaire,Bordeaux,NaN,Bordeaux,France,...,NaN,Bordeaux,Red,750,Unknown,4,3,Casual,Still Red,Bolaire Château Bolaire 2016 75.0 40261
1,54770,3,1,75.0,Ygrec de Ch. d'Yquem,Château d'Yquem,Bordeaux,NaN,Bordeaux,France,...,NaN,Bordeaux,White,750,Unknown,2,3,Dinner,Still White,Ygrec de Ch. d'Yquem Château d'Yquem 2020 75.0...
2,62182,108,6,75.0,Ygrec de Ch. d'Yquem,Château d'Yquem,Bordeaux,NaN,Bordeaux,France,...,22.0,Bordeaux,White,750,Unknown,2,3,Dinner,Still White,Ygrec de Ch. d'Yquem Château d'Yquem 2022 75.0...


In [7]:
# --- CELL 6: CPI Calculation + Filter Wiring (Clean & Robust) ---
# Honors UI filters (loyalty, wine type, bottle size, price tier, last stock, seasonality),
# computes CPI against stock, and saves compact outputs the UI can reuse.

import os
import json
import numpy as np
import pandas as pd
from datetime import datetime
from pathlib import Path
from time import perf_counter

# ---------- Helpers ----------
def get_season_from_week(week_no: int):
    if 1 <= week_no <= 8 or 49 <= week_no <= 53: return "Winter"
    if 9  <= week_no <= 22: return "Spring"
    if 23 <= week_no <= 35: return "Summer"
    if 36 <= week_no <= 48: return "Autumn"
    return "Unknown"

def extract_ids_from_weekly_calendar(obj) -> set:
    ids = set()
    if isinstance(obj, dict):
        for arr in obj.values():
            if isinstance(arr, list):
                for it in arr:
                    if isinstance(it, dict):
                        v = it.get("id") or it.get("wine_id")
                        if v is not None and str(v).strip():
                            ids.add(str(v))
    elif isinstance(obj, list):
        for it in obj:
            if isinstance(it, dict):
                v = it.get("id") or it.get("wine_id")
                if v is not None and str(v).strip():
                    ids.add(str(v))
    return ids

def coerce_numeric(s, default=np.nan):
    try:
        return pd.to_numeric(s, errors="coerce")
    except Exception:
        return default

# keep the calendar_year chosen earlier; do NOT reset to now().year later
try:
    week_number = int(week_number)
except Exception:
    week_number = int(os.getenv("WEEK_NUMBER", datetime.now().isocalendar().week))
try:
    year = int(calendar_year)
except Exception:
    year = datetime.now().year

selected_season = get_season_from_week(week_number)

# text-mode tqdm (avoid widget deps)
try:
    from tqdm import tqdm
except Exception:
    def tqdm(x, **k): return x

# ---------- Ensure inputs are present ----------
# If notebook was run standalone, try loading from previous cell outputs
if 'clients_df' not in globals():
    p = OUTPUT_PATH / "filtered_clients.pkl"
    clients_df = pd.read_pickle(p) if p.exists() else pd.DataFrame()
if 'stock_df' not in globals():
    p = OUTPUT_PATH / "stock_df_final.pkl"
    stock_df = pd.read_pickle(p) if p.exists() else pd.DataFrame()

# Normalize schemas
clients_df = clients_df.copy()
clients_df.columns = (
    clients_df.columns
    .str.strip().str.lower().str.replace(" ", "_").str.replace(".", "", regex=False)
)
if 'customer_no' in clients_df.columns:
    clients_df['customer_no'] = clients_df['customer_no'].astype(str).str.strip()
if 'item_no' in clients_df.columns:
    clients_df['item_no'] = clients_df['item_no'].astype(str).str.strip()

stock_df = stock_df.copy()
if 'id' in stock_df.columns:
    stock_df['id'] = stock_df['id'].astype(str).str.strip()

# critic score column consistency
if 'avg_score' not in stock_df.columns and 'AVG' in stock_df.columns:
    stock_df.rename(columns={'AVG': 'avg_score'}, inplace=True)

# ---------- Loyalty derivation (counts → tier) ----------
if {'customer_no','item_no'}.issubset(clients_df.columns):
    purchase_counts = clients_df.groupby('customer_no')['item_no'].count().reset_index(name='purchase_count')
    purchase_counts['loyalty_level'] = purchase_counts['purchase_count'].apply(
        lambda x: 'vip' if x >= 20 else 'gold' if x >= 10 else 'silver' if x >= 5 else 'bronze'
    )
    clients_df = clients_df.drop(columns=['loyalty_level'], errors='ignore') \
                           .merge(purchase_counts[['customer_no','loyalty_level']], on='customer_no', how='left')
else:
    clients_df['loyalty_level'] = 'bronze'
clients_df['loyalty_level'] = clients_df['loyalty_level'].fillna('bronze')

# ---------- Bottle size ML on clients (fallback only) ----------
if 'bottle_size' in clients_df.columns:
    def _to_ml(x):
        try:
            v = float(x)
            return round(v*10) if v < 100 else v
        except:
            return np.nan
    clients_df['bottle_size_ml'] = clients_df['bottle_size'].apply(_to_ml)
else:
    clients_df['bottle_size_ml'] = np.nan

# Optional sales date
sales_dates = clients_df[['customer_no','sales_date']].drop_duplicates() \
    if 'sales_date' in clients_df.columns else pd.DataFrame(columns=['customer_no','sales_date'])

# Vintage from stock by item_no (if available)
if 'item_no' in clients_df.columns and 'vintage' in stock_df.columns and 'id' in stock_df.columns:
    stock_vintage_map = stock_df.set_index('id')['vintage'].astype(str).to_dict()
    clients_df['vintage'] = clients_df['item_no'].map(stock_vintage_map)
else:
    clients_df['vintage'] = 'Unknown'

def classify_vintage_group(vint):
    try:
        s = str(vint).strip().upper()
        if s == "NV": return "Non-Vintage Lover"
        y = int(s); cy = datetime.now().year
        if y == cy: return "Likes Current Vintage"
        if y == cy-1: return "Likes En Primeur"
        if cy - y <= 3: return "Likes Young Wines"
        if cy - y <= 8: return "Likes Mature Wines"
        if cy - y > 15: return "Likes Old Wines"
        return "Unknown"
    except:
        return "Unknown"

clients_df['inferred_vintage'] = clients_df['vintage'].apply(classify_vintage_group)

def classify_size_from_ml(size_ml):
    try:
        v = int(float(size_ml))
        m = {375:"Half",750:"Standard",1500:"Magnum",3000:"Jeroboam",4500:"Rehoboam",
             6000:"Methuselah",9000:"Salmanazar",12000:"Balthazar",15000:"Nebuchadnezzar",
             18000:"Melchior",27000:"Primat"}
        return m.get(v, "Unknown")
    except:
        return "Unknown"

# ---------- Build purchases_df for preference inference ----------
needed = ['id','grape_list','type','region','sweetness','body','price_tier','avg_score','vintage','bottle_size_ml']
stock_merge_df = stock_df.copy()
for c in needed:
    if c not in stock_merge_df.columns:
        stock_merge_df[c] = np.nan

if 'item_no' not in clients_df.columns:
    purchases_df = clients_df.copy()
    for c in ['grape_list','type','region','sweetness','body','price_tier','avg_score','vintage','bottle_size_ml']:
        purchases_df[c] = np.nan
else:
    purchases_df = clients_df.merge(
        stock_merge_df[needed], left_on='item_no', right_on='id', how='left'
    )

purchases_df['vintage'] = purchases_df.get('vintage', pd.Series(["Unknown"]*len(purchases_df))).fillna("Unknown")
if 'bottle_size_ml' not in purchases_df.columns:
    purchases_df['bottle_size_ml'] = 750
purchases_df['size_group'] = purchases_df['bottle_size_ml'].apply(classify_size_from_ml)

# Occasion inference (coarse)
def infer_occasion(row):
    try:
        price = str(row.get('price_tier','')).lower()
        wine_type = str(row.get('type','')).lower()
        size = float(row.get('bottle_size_ml',750))
        score = float(row.get('avg_score',0))
        sweetness = float(row.get('sweetness',0))
        if size >= 3000: return 'Big Event'
        if size >= 1500: return 'Celebration'
        if 'sparkling' in wine_type or sweetness >= 4: return 'Celebration'
        if score >= 92 or (price in ['luxury','ultra luxury','premium'] and size == 750): return 'Gift'
        if price in ['budget','entry'] and size <= 750 and sweetness <= 2: return 'Everyday'
        if price in ['mid-range','premium'] and 'still' in wine_type: return 'Dinner'
        if score >= 90 and price in ['mid-range','premium']: return 'Dinner'
        if sweetness <= 2 and score >= 88 and size <= 750: return 'Everyday'
        if price in ['mid-range','premium','luxury'] and 'still' in wine_type: return 'Personal Consumption'
        return 'Unknown'
    except:
        return 'Unknown'
purchases_df['occasion'] = purchases_df.apply(infer_occasion, axis=1)

# ---------- Preference aggregation ----------
def split_and_flatten(series, delimiter='/'):
    return [x.strip() for s in series.dropna() for x in str(s).split(delimiter) if x.strip()]

inferred_pref = purchases_df.groupby('customer_no').agg({
    'grape_list':  lambda x: ','.join(sorted(set(split_and_flatten(x)))),
    'type':        lambda x: ','.join(sorted(set(x.dropna().astype(str).str.lower()))),
    'region':      lambda x: ','.join(sorted(set(x.dropna().astype(str).str.lower()))),
    'sweetness':   lambda x: round(coerce_numeric(x).dropna().astype(float).mean(), 2) if len(x.dropna()) else np.nan,
    'body':        lambda x: round(coerce_numeric(x).dropna().astype(float).mean(), 2) if len(x.dropna()) else np.nan,
    'price_tier':  lambda x: x.dropna().mode().iloc[0] if not x.dropna().mode().empty else '',
    'avg_score':   lambda x: round(coerce_numeric(x).dropna().astype(float).mean(), 2) if len(x.dropna()) else np.nan,
    'vintage':     lambda x: ','.join(sorted(set(x.dropna().astype(str)))),
    'size_group':  lambda x: ','.join(sorted(set(x.dropna().astype(str)))),
    'occasion':    lambda x: ','.join(sorted(set(x.dropna().astype(str))))
}).reset_index().rename(columns={
    'grape_list':'inferred_grape_preferences',
    'type':'inferred_type',
    'region':'inferred_region',
    'sweetness':'inferred_sweetness',
    'body':'inferred_body',
    'price_tier':'inferred_budget',
    'avg_score':'avg_critic_score',
    'vintage':'inferred_vintage',
    'size_group':'inferred_size',
    'occasion':'inferred_occasion'
})

# Fallback vintage/size preferences when inference is missing
clients_df['fallback_size_label'] = clients_df['bottle_size_ml'].apply(classify_size_from_ml)
fallback_vintage = (
    clients_df.groupby('customer_no')['vintage']
    .agg(lambda x: x.dropna().mode().iloc[0] if not x.dropna().mode().empty else 'Unknown')
    .reset_index(name='fallback_vintage')
)
fallback_vintage['fallback_vintage_label'] = fallback_vintage['fallback_vintage'].apply(classify_vintage_group)

inferred_pref = inferred_pref.merge(
    fallback_vintage[['customer_no','fallback_vintage_label']], on='customer_no', how='left'
)
inferred_pref['inferred_vintage'] = inferred_pref['fallback_vintage_label'].fillna(inferred_pref['inferred_vintage'])
inferred_pref.drop(columns=['fallback_vintage_label'], inplace=True)

inferred_pref = inferred_pref.merge(
    clients_df[['customer_no','fallback_size_label']].drop_duplicates(),
    on='customer_no', how='left'
)
inferred_pref['inferred_size'] = inferred_pref['fallback_size_label'].fillna(inferred_pref['inferred_size'])
inferred_pref.drop(columns=['fallback_size_label'], inplace=True)

# Merge traits + loyalty + sales date
client_pref_df = (inferred_pref
    .merge(clients_df[['customer_no']].drop_duplicates(), on='customer_no', how='right')
    .merge(clients_df[['customer_no','loyalty_level']].drop_duplicates(), on='customer_no', how='left')
    .merge(sales_dates, on='customer_no', how='left')
)
client_pref_df['loyalty_level'] = client_pref_df['loyalty_level'].fillna('bronze')

# Flags
client_pref_df['prefers_high_scores'] = (coerce_numeric(client_pref_df['avg_critic_score']).fillna(0) >= 95)
stock_df['avg_score'] = coerce_numeric(stock_df.get('avg_score', np.nan))
stock_df['high_score'] = stock_df['avg_score'].ge(95)

# ---------- UI Filters (from Cell 1 or env) ----------
if 'filters' not in globals():
    try:
        filters = json.loads(os.getenv("FILTER_INPUTS", "{}"))
    except Exception:
        filters = {}

# Loyalty filter (client subset)
loy = str(filters.get('loyalty','all') or 'all').lower()
if loy != 'all':
    before = len(client_pref_df)
    client_pref_df = client_pref_df.loc[client_pref_df['loyalty_level'].str.lower() == loy].copy()
    after = len(client_pref_df)
    print(f"🎯 Loyalty filter [{loy}] → clients: {before} → {after}")

# Wine type filter (stock subset)
wt = (filters.get('wine_type') or '').strip().lower()
if wt:
    if wt in {'red','white','rose','rosé','sparkling','dessert'}:
        if wt in {'rose','rosé'}:
            mask = stock_df.get('color','').astype(str).str.contains('ros', case=False, na=False)
        elif wt == 'sparkling':
            mask = stock_df.get('type','').astype(str).str.contains('sparkling', case=False, na=False) \
                   | stock_df.get('color','').astype(str).str.contains('spark', case=False, na=False)
        elif wt == 'dessert':
            mask = stock_df.get('type','').astype(str).str.contains('dessert', case=False, na=False)
        else:
            mask = stock_df.get('color','').astype(str).str.contains(wt, case=False, na=False)
        before = len(stock_df)
        stock_df = stock_df.loc[mask].copy()
        after = len(stock_df)
        print(f"🍷 Wine type [{wt}] → wines: {before} → {after}")

# Bottle size filter
bs = filters.get('bottle_size', None)
try:
    bs_ml = int(float(bs)) if bs is not None else None
except Exception:
    bs_ml = None
if bs_ml:
    before = len(stock_df)
    stock_df = stock_df.loc[(coerce_numeric(stock_df.get('bottle_size_ml')).round(0) == bs_ml)].copy()
    after = len(stock_df)
    print(f"🍾 Bottle size [{bs_ml}ml] → wines: {before} → {after}")

# Price tier bucket filter
pt = (filters.get('price_tier_bucket') or '').strip()
if pt:
    before = len(stock_df)
    stock_df = stock_df.loc[stock_df.get('price_tier','').astype(str).str.lower() == pt.lower()].copy()
    after = len(stock_df)
    print(f"💰 Price tier [{pt}] → wines: {before} → {after}")

# Low stock filter
if filters.get('last_stock', False):
    th = int(filters.get('last_stock_threshold', 10) or 10)
    before = len(stock_df)
    stock_df = stock_df.loc[coerce_numeric(stock_df.get('stock')).fillna(0) < th].copy()
    after = len(stock_df)
    print(f"⚠️ Last stock < {th} → wines: {before} → {after}")

# Seasonality filter (only if column exists)
if filters.get("seasonality_boost", False) and "seasonality_boost" in stock_df.columns:
    stock_df["seasonal_match"] = stock_df["seasonality_boost"].apply(
        lambda x: selected_season in x if isinstance(x, list) else False
    )
    before = len(stock_df)
    stock_df = stock_df.loc[stock_df["seasonal_match"] == True].copy()
    after = len(stock_df)
    print(f"🌿 Seasonality [{selected_season}] → wines: {before} → {after}")
elif filters.get("seasonality_boost", False):
    print("⚠️ 'seasonality_boost' column not found — skipping seasonality filter")

# Recency filter (avoid using items that were already used last year in same week, or this week if already emitted)
def _load_calendar_ids(p: Path) -> set:
    try:
        if p.suffix.lower() == ".pkl":
            obj = pd.read_pickle(p)
            cal = obj.get("weekly_calendar", obj) if isinstance(obj, dict) else obj
        else:
            cal = json.loads(p.read_text(encoding="utf-8"))
            cal = cal.get("weekly_calendar", cal)
        return extract_ids_from_weekly_calendar(cal)
    except Exception:
        return set()

candidates = [
    OUTPUT_PATH / f"weekly_campaign_schedule_{year-1}_week_{week_number}.pkl",
    OUTPUT_PATH / f"weekly_campaign_schedule_{year-1}_week_{week_number}.json",
    OUTPUT_PATH / f"weekly_campaign_schedule_{year}_week_{week_number}.pkl",
    OUTPUT_PATH / f"weekly_campaign_schedule_{year}_week_{week_number}.json",
    OUTPUT_PATH / f"weekly_campaign_schedule_week_{week_number}.pkl",   # legacy
    OUTPUT_PATH / f"weekly_campaign_schedule_week_{week_number}.json",   # legacy
]
past_ids = set()
used_path = None
for p in candidates:
    if p.exists():
        ids = _load_calendar_ids(p)
        if ids:
            past_ids = ids
            used_path = p
            break

if past_ids:
    before = len(stock_df)
    stock_df = stock_df.loc[~stock_df['id'].astype(str).isin(past_ids)].copy()
    after = len(stock_df)
    print(f"⏮️ Recency filter → removed {before - after} (source: {used_path.name})")
else:
    print(f"📁 No past schedule for Week {week_number} → skipping recency filter")

# Refresh flags after filters
stock_df['avg_score'] = coerce_numeric(stock_df.get('avg_score'))
stock_df['high_score'] = stock_df['avg_score'].ge(95)

# ---------- CPI compute ----------
display_col = 'wine' if 'wine' in stock_df.columns else 'id'

def compute_cpi_matrix(client_df, stock_df, style="default", display_col='wine'):
    stock_df = stock_df.copy()
    stock_df['id'] = stock_df['id'].astype(str)

    # Baseline weights
    weights = {
        'grape': 1.0, 'type': 1.0, 'region': 1.0,
        'sweetness': 0.5, 'body': 0.5,
        'budget': 0.75, 'prefers_high_scores': 0.75,
        'avg_score': 0.5
    }
    loyalty_bonus = {'bronze': 0.0, 'silver': 0.25, 'gold': 0.5, 'vip': 0.75}

    # Persona tweaks
    st = (style or 'default').lower()
    if st == "cat":
        weights['type'] = 1.2
        weights['region'] = 1.2
        weights['avg_score'] = 0.4
    elif st == "nigo":
        weights['budget'] = 1.0
        weights['avg_score'] = 0.7

    total_possible = sum(weights.values()) + max(loyalty_bonus.values())

    # Ensure numeric for tolerance checks
    sd_sweet = coerce_numeric(stock_df.get('sweetness')).fillna(-999)
    sd_body  = coerce_numeric(stock_df.get('body')).fillna(-999)

    result = []
    it = client_df.iterrows() if len(client_df) else iter([(-1, {'customer_no':'GLOBAL','inferred_grape_preferences':'','inferred_type':'','inferred_region':'','inferred_sweetness':np.nan,'inferred_body':np.nan,'inferred_budget':'','avg_critic_score':np.nan,'loyalty_level':'bronze','prefers_high_scores':False})])

    for _, client in tqdm(it, total=max(len(client_df),1), desc="🔄 Generating CPI vectors"):
        score = pd.Series(0.0, index=stock_df.index)

        # grapes
        grape_prefs = set(str(client.get('inferred_grape_preferences','')).lower().split(','))
        wine_grapes = stock_df.get('grape_list','').fillna('').astype(str).str.lower().str.split('/')
        score += wine_grapes.apply(lambda gs: any(g.strip() in grape_prefs for g in gs if g)).astype(float) * weights['grape']

        # type/region
        score += (stock_df.get('type','').fillna('').astype(str).str.lower()
                  .str.contains(str(client.get('inferred_type','')).lower(), na=False)) * weights['type']
        score += (stock_df.get('region','').fillna('').astype(str).str.lower()
                  .str.contains(str(client.get('inferred_region','')).lower(), na=False)) * weights['region']

        # sweetness/body (tolerance)
        cs = client.get('inferred_sweetness', np.nan)
        cb = client.get('inferred_body', np.nan)
        if pd.notna(cs):
            score += (np.isclose(sd_sweet, float(cs), atol=0.5)).astype(float) * weights['sweetness']
        if pd.notna(cb):
            score += (np.isclose(sd_body,  float(cb), atol=0.5)).astype(float) * weights['body']

        # budget match
        score += (stock_df.get('price_tier','').fillna('').astype(str).str.lower()
                  == str(client.get('inferred_budget','')).lower()) * weights['budget']

        # high score preference
        pref_hi = bool(client.get('prefers_high_scores', False))
        score += (stock_df['high_score'] & pref_hi).astype(float) * weights['prefers_high_scores']

        # general avg score quality
        score += (coerce_numeric(stock_df['avg_score']).fillna(0) >= 90).astype(float) * weights['avg_score']

        # loyalty
        score += loyalty_bonus.get(str(client.get('loyalty_level','bronze')).lower(), 0)

        score /= total_possible
        result.append(score.round(4).rename(f"pref_cpi_for_{client.get('customer_no','GLOBAL')}"))

    matrix = pd.concat([stock_df[['id', display_col]].reset_index(drop=True)] + result, axis=1)
    return matrix

style = (filters.get('style') or 'default').lower()
t0 = perf_counter()
cpi_matrix = compute_cpi_matrix(client_pref_df, stock_df, style=style, display_col=display_col)
print("⏱️ CPI computation completed in", round(perf_counter() - t0, 2), "seconds.")

# ---------- Attach average CPI per wine BEFORE saving UI snapshot ----------
cpi_cols = [c for c in cpi_matrix.columns if c.startswith("pref_cpi_for_")]
if cpi_cols:
    cpi_avg_df = pd.DataFrame({
        "id": cpi_matrix["id"].astype(str),
        "avg_cpi_score": cpi_matrix[cpi_cols].mean(axis=1).round(4)
    })
    stock_df = stock_df.merge(cpi_avg_df, on="id", how="left")
else:
    stock_df["avg_cpi_score"] = np.nan

# ---------- Save outputs ----------
client_pref_df.to_pickle(OUTPUT_PATH / "client_pref_df_latest.pkl")
cpi_matrix.to_pickle(OUTPUT_PATH / "cpi_matrix_latest.pkl")

# a compact stock file the webapp can use to render cards
ui_cols = [
    'id','wine','producer','vintage','price_tier','stock','full_type','type','color',
    'region','region_group','bottle_size_ml','avg_score','high_score','avg_cpi_score'
]
present = [c for c in ui_cols if c in stock_df.columns]
stock_df[present].to_pickle(OUTPUT_PATH / "stock_for_ui_latest.pkl")

print("✅ Preferences, CPI matrix, and UI stock snapshot saved (.pkl).")
print("🧪 CPI Matrix shape:", cpi_matrix.shape, "| UI stock cols:", present)


🍾 Bottle size l] → wines: 4447 → 2342
⚠️ Last stock < 10 → wines: 2342 → 756
⏮️ Recency filter → removed 25 (source: weekly_campaign_schedule_2025_week_36.pkl)


🔄 Generating CPI vectors:   0%|                                            | 0/1219 [00:00<?, ?it/s]

🔄 Generating CPI vectors:   1%|▎                                 | 12/1219 [00:00<00:10, 115.38it/s]

🔄 Generating CPI vectors:   2%|▋                                 | 24/1219 [00:00<00:11, 101.74it/s]

🔄 Generating CPI vectors:   3%|▉                                 | 35/1219 [00:00<00:11, 100.16it/s]

...

🔄 Generating CPI vectors:  98%|███████████████████████████████▎| 1193/1219 [00:08<00:00, 139.53it/s]

🔄 Generating CPI vectors:  99%|███████████████████████████████▋| 1209/1219 [00:09<00:00, 140.71it/s]

🔄 Generating CPI vectors: 100%|████████████████████████████████| 1219/1219 [00:09<00:00, 132.94it/s]

⏱️ CPI computation completed in 9.61 seconds.


✅ Preferences, CPI matrix, and UI stock snapshot saved (.pkl).
🧪 CPI Matrix shape: (1422, 1221) | UI stock cols: ['id', 'wine', 'producer', 'vintage', 'price_tier', 'stock', 'full_type', 'type', 'color', 'region', 'region_group', 'bottle_size_ml', 'avg_score', 'high_score', 'avg_cpi_score']


In [8]:
# --- CELL 7: Build/refresh fallback_pool safely ---
# Goal: give the scheduler a good "pool" to draw from when client prefs can't fill the week.

import os, json
from pathlib import Path
import pandas as pd
import numpy as np
from IPython.display import display

# ---------- Fallbacks if prior cells didn't run ----------
try:
    OUTPUT_PATH
except NameError:
    _default_output = Path.home() / "OneDrive - AVU SA" / "AVU CPI Campaign" / "Puzzle_control_Reports" / "IRON_DATA"
    OUTPUT_PATH = Path(os.getenv("OUTPUT_PATH", _default_output))
    OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

if 'stock_df' not in globals():
    _stock_pkl = OUTPUT_PATH / "stock_df_final.pkl"
    stock_df = pd.read_pickle(_stock_pkl) if _stock_pkl.exists() else pd.DataFrame()

if 'id' not in stock_df.columns:
    stock_df['id'] = ""  # ensure merge key exists

# ---------- Guards: ensure filters + UI vars exist even if Cell 5/6 didn't run ----------
# 1) Ensure `filters` is a dict (load from file/env if missing)
if 'filters' not in globals() or not isinstance(filters, dict):
    filters = {}
    _filters_path = os.path.join("notebooks", "filters.json")
    if os.path.exists(_filters_path):
        try:
            with open(_filters_path, "r", encoding="utf-8") as f:
                filters = json.load(f) or {}
        except Exception as _e:
            print(f"⚠️ Could not read filters.json: {_e}")
    else:
        try:
            env_f = os.getenv("FILTER_INPUTS", "")
            if env_f:
                filters = json.loads(env_f)
        except Exception:
            pass

# 2) Define UI variables with robust defaults
if 'selected_type' not in globals():
    selected_type = filters.get("wine_type", None)

if 'selected_size' not in globals():
    _sz = filters.get("bottle_size", None)
    try:
        selected_size = int(_sz) if _sz is not None and str(_sz).strip().lower() != "bigger" else None
    except Exception:
        selected_size = None

if 'last_stock' not in globals():
    _ls = filters.get("last_stock", False)
    last_stock = (str(_ls).strip().lower() in ("1", "true", "yes", "y"))

if 'seasonality_boost' not in globals():
    _sb = filters.get("seasonality_boost", False)
    seasonality_boost = (str(_sb).strip().lower() in ("1", "true", "yes", "y"))

def _parse_size_to_ml(val):
    try:
        s = str(val).strip().lower().replace("ml","").replace("cl","").replace("l","")
        if not s: return np.nan
        v = float(s)
        if v <= 100:  # cl
            return int(round(v * 10))
        if v < 20:    # liters
            return int(round(v * 1000))
        return int(round(v))  # already ml
    except Exception:
        return np.nan

# ---------- Try CPI-augmented pool; else fall back to stock only ----------
try:
    if "merged_cpi_df" in globals():
        _cpi_src = merged_cpi_df.copy()
        cpi_cols = [c for c in _cpi_src.columns if c.startswith("pref_cpi_for_")]
        _avg = _cpi_src[cpi_cols].mean(axis=1) if cpi_cols else pd.Series(np.nan, index=_cpi_src.index)
        cpi_avg_df = _cpi_src[['id']].copy()
        cpi_avg_df['avg_cpi'] = _avg
    else:
        _cpi_path = OUTPUT_PATH / "cpi_matrix_latest.pkl"
        _cpi = pd.read_pickle(_cpi_path) if _cpi_path.exists() else pd.DataFrame()
        if _cpi.empty or 'id' not in _cpi.columns:
            raise FileNotFoundError("cpi_matrix_latest.pkl missing or malformed")
        cpi_cols = [c for c in _cpi.columns if c.startswith("pref_cpi_for_")]
        _avg = _cpi[cpi_cols].mean(axis=1) if cpi_cols else pd.Series(np.nan, index=_cpi.index)
        cpi_avg_df = pd.DataFrame({"id": _cpi['id'].astype(str), "avg_cpi": _avg.values})

    # Assemble base pool from stock with useful columns
    _stock_cols = ['id','wine','vintage','stock','avg_score','price_tier','full_type','region_group']
    have_cols = [c for c in _stock_cols if c in stock_df.columns]
    base = stock_df[have_cols].copy()
    base['id'] = base['id'].astype(str)

    # Bottle size to ML (prefer existing column)
    if 'bottle_size_ml' in stock_df.columns:
        base = base.merge(stock_df[['id','bottle_size_ml']], on='id', how='left')
    else:
        size_sources = [c for c in stock_df.columns if c.lower() in ('size','size_cl','bottle_size','bottle size')]
        if size_sources:
            _sz = stock_df[['id', size_sources[0]]].rename(columns={size_sources[0]: 'size_raw'})
            _sz['bottle_size_ml'] = _sz['size_raw'].apply(_parse_size_to_ml)
            base = base.merge(_sz[['id','bottle_size_ml']], on='id', how='left')
        else:
            base['bottle_size_ml'] = np.nan

    # Attach CPI average
    fallback_pool = base.merge(cpi_avg_df, on='id', how='left')

except Exception as e:
    print(f"⚠️ Could not build CPI-based fallback pool: {e}")
    base_cols = ['id','wine','vintage','stock','avg_score','price_tier','full_type','region_group']
    have_cols = [c for c in base_cols if c in stock_df.columns]
    if 'id' not in have_cols:
        have_cols = ['id'] + have_cols
    fallback_pool = stock_df[have_cols].copy()
    if 'bottle_size_ml' in stock_df.columns and 'bottle_size_ml' not in fallback_pool.columns:
        fallback_pool = fallback_pool.merge(stock_df[['id','bottle_size_ml']], on='id', how='left')
    fallback_pool['avg_cpi'] = np.nan  # ensure column exists

# ---------- Ensure dtypes ----------
fallback_pool['stock']     = pd.to_numeric(fallback_pool.get('stock', 0), errors='coerce').fillna(0).astype(int)
fallback_pool['avg_cpi']   = pd.to_numeric(fallback_pool.get('avg_cpi', np.nan), errors='coerce')
fallback_pool['avg_score'] = pd.to_numeric(fallback_pool.get('avg_score', np.nan), errors='coerce')

# If CPI avg came from Cell 6 as avg_cpi_score, use it
if ('avg_cpi' not in fallback_pool.columns) or fallback_pool['avg_cpi'].isna().all():
    if 'avg_cpi_score' in stock_df.columns:
        try:
            fallback_pool = fallback_pool.merge(
                stock_df[['id','avg_cpi_score']].rename(columns={'avg_cpi_score':'avg_cpi'}),
                on='id', how='left'
            )
        except Exception:
            pass

# ---------- Apply UI filters ----------
_selected_type = (selected_type or "").strip()
_selected_size = selected_size
_last_stock    = bool(last_stock)
_season_boost  = bool(seasonality_boost)

# wine type
if _selected_type and _selected_type.lower() != "all":
    if 'full_type' in fallback_pool.columns:
        mask = fallback_pool['full_type'].astype(str).str.contains(_selected_type, case=False, na=False)
        fallback_pool = fallback_pool[mask]
    else:
        mask = False
        for c in [c for c in fallback_pool.columns if c.lower() in ('type','color','full_type')]:
            mask = mask | fallback_pool[c].astype(str).str.contains(_selected_type, case=False, na=False)
        fallback_pool = fallback_pool[mask]

# bottle size (ml)
if _selected_size:
    try:
        sel_ml = int(_selected_size)
        fallback_pool = fallback_pool[ fallback_pool['bottle_size_ml'].fillna(-1).astype(int) == sel_ml ]
    except Exception:
        pass

# last stock (<10)
if _last_stock:
    fallback_pool = fallback_pool[fallback_pool['stock'] < 10]

# seasonality boost (only if column exists and is truthy)
if _season_boost and 'seasonality_boost' in stock_df.columns:
    season_ids = set(
        stock_df.loc[
            stock_df['seasonality_boost'].apply(lambda x: bool(x) and str(x).strip() not in ('[]','False','false','0')),
            'id'
        ].astype(str)
    )
    fallback_pool = fallback_pool[fallback_pool['id'].astype(str).isin(season_ids)]

print(f"🧮 Fallback pool after filters: {len(fallback_pool)} rows")

# ---------- Rank (prefer higher CPI, then score, then stock) ----------
if 'avg_cpi' not in fallback_pool.columns:
    fallback_pool['avg_cpi'] = np.nan
fallback_pool['avg_cpi']   = pd.to_numeric(fallback_pool['avg_cpi'], errors='coerce')
fallback_pool['avg_score'] = pd.to_numeric(fallback_pool['avg_score'], errors='coerce')

fallback_pool = fallback_pool.sort_values(
    ['avg_cpi','avg_score','stock'], ascending=[False, False, False], na_position='last'
).reset_index(drop=True)

# Persist for the webapp/scheduler
fallback_path = OUTPUT_PATH / "fallback_pool.pkl"
fallback_pool.to_pickle(fallback_path)
print(f"💾 Saved fallback_pool → {fallback_path} (top {min(5, len(fallback_pool))} rows):")
display(fallback_pool.head(5))


🧮 Fallback pool after filters: 731 rows
💾 Saved fallback_pool → C:\Users\Marco.Africani\OneDrive - AVU SA\AVU CPI Campaign\Puzzle_control_Reports\IRON_DATA\fallback_pool.pkl (top 5 rows):


,id,wine,vintage,stock,avg_score,price_tier,full_type,region_group,bottle_size_ml,avg_cpi
0,62139,Cabernet Sauvignon Pritchard Hill,2021,3,97.000000,Luxury,Still Red,Napa Valley,750,0.535446
1,57376,Cabernet Sauvignon Helena Montana,2018,3,95.000000,Premium,Still Red,Napa Valley,750,0.535446
2,60057,Melbury,2020,3,95.000000,Ultra Luxury,Still Red,Napa Valley,750,0.535446
3,60582,La Gaffelière,2023,6,94.846154,Mid-range,Still Red,St. Emilion,750,0.535446
4,64332,Le Pin,2024,9,94.607143,Ultra Luxury,Still Red,Pomerol,750,0.535446


In [9]:
# ---  CELL 8: Save UI-Ready JSON & PKL for Flask ---

import os
import json
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from pathlib import Path
from time import time
start_time = time()

# ---------- Fallbacks if earlier cells didn't run ----------
try:
    OUTPUT_PATH
except NameError:
    _default_output = Path.home() / "OneDrive - AVU SA" / "AVU CPI Campaign" / "Puzzle_control_Reports" / "IRON_DATA"
    OUTPUT_PATH = Path(os.getenv("OUTPUT_PATH", _default_output))
    OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

try:
    NUM_SLOTS
except NameError:
    NUM_SLOTS = 5

# Load stock if not in memory
if 'stock_df' not in globals():
    _stock_pkl = OUTPUT_PATH / "stock_df_final.pkl"
    stock_df = pd.read_pickle(_stock_pkl) if _stock_pkl.exists() else pd.DataFrame()

# Load client pref if not in memory
if 'client_pref_df' not in globals():
    _pref_pkl = OUTPUT_PATH / "client_pref_df_latest.pkl"
    client_pref_df = pd.read_pickle(_pref_pkl) if _pref_pkl.exists() else pd.DataFrame(columns=['customer_no'])

# ---------- Locked snapshot helpers ----------
DAYS = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]

def _load_json(path: Path):
    try:
        if path.exists() and path.stat().st_size > 0:
            return json.loads(path.read_text(encoding="utf-8"))
    except Exception:
        pass
    return None

def _locked_to_df(locked_json: dict, stock_df: pd.DataFrame) -> pd.DataFrame:
    rows = []
    for day in DAYS:
        for it in (locked_json.get(day) or []):
            if not it: 
                continue
            rows.append({"day": day, **it})
    df = pd.DataFrame(rows)
    if df.empty:
        return df

    df["id"] = df.get("id","").astype(str)
    enrich_cols = ["id","wine","vintage","full_type","region_group","stock","price_tier","avg_cpi_score"]
    have = [c for c in enrich_cols if c in stock_df.columns]
    if have:
        df = df.merge(stock_df[have].drop_duplicates("id"), on="id", how="left")
    df["locked"] = True
    # Title-case price_tier if present; otherwise create an empty column
    if "price_tier" in df.columns:
        df["price_tier"] = df["price_tier"].apply(
            lambda x: x.title().strip() if isinstance(x, str) and x else ""
        )
    else:
        df["price_tier"] = ""
    return df


# ---------- Determine selected year/week and load locked snapshot ----------
try:
    sel_year = int(calendar_year)
except Exception:
    sel_year = datetime.now().year
try:
    sel_week = int(week_number)
except Exception:
    sel_week = datetime.now().isocalendar().week

LOCKED_PATH = OUTPUT_PATH / "locked_weeks"
LOCKED_PATH.mkdir(parents=True, exist_ok=True)

persisted = _load_json(LOCKED_PATH / f"locked_calendar_{sel_year}_week_{sel_week}.json")

# Prefer in-memory structured snapshot from Cell 1; else persisted; else empty
if "locked_calendar_snapshot" in globals() and locked_calendar_snapshot:
    # Convert structured snapshot {day:{main,overflow}} → flat lists per day
    locked_json = {d: (locked_calendar_snapshot.get(d, {}).get("main") or []) for d in DAYS}
elif persisted:
    locked_json = persisted
else:
    locked_json = {d: [] for d in DAYS}

# Now that we have locked_json and stock_df, build locked_df
locked_df = _locked_to_df(locked_json, stock_df)

# ---------- Tier helpers ----------
TIER_MAP_NAME = {
    "budget": "Budget",
    "entry": "Budget",
    "mid": "Mid-range",
    "mid_range": "Mid-range",
    "mid-range": "Mid-range",
    "premium": "Premium",
    "luxury": "Luxury",
    "ultra": "Ultra Luxury",
    "ultra-luxury": "Ultra Luxury",
    "ultra_luxury": "Ultra Luxury",
    "ultra luxury": "Ultra Luxury",
}
TIER_MAP_ID = {
    "Budget": "budget",
    "Mid-range": "mid",
    "Premium": "premium",
    "Luxury": "luxury",
    "Ultra Luxury": "ultra",
}

def canon_tier_name(x: str) -> str:
    s = str(x or "").strip()
    if not s:
        return ""
    key = s.lower().replace(" ", "_").replace("__", "_").replace("-", "_")
    return TIER_MAP_NAME.get(key, s)

def tier_id_from_name(name: str) -> str:
    return TIER_MAP_ID.get(canon_tier_name(name), "")

def to_ml(size_val):
    """Accepts '750', '75cl', '1.5L', 750 etc. -> returns int ml or np.nan"""
    if size_val is None:
        return np.nan
    s = str(size_val).strip().lower()
    try:
        if s.endswith("ml"):
            return int(float(s[:-2]))
        if s.endswith("cl"):
            return int(float(s[:-2]) * 10)
        if s.endswith("l"):
            return int(float(s[:-1]) * 1000)
        # plain number: if < 100 -> assume cl, else ml
        v = float(s)
        return int(v * 10) if v < 100 else int(v)
    except Exception:
        return np.nan

def ensure_cols(df: pd.DataFrame, cols: dict):
    """Ensure columns exist with default; cols = {name: default}"""
    for c, default in cols.items():
        if c not in df.columns:
            df[c] = default
    return df

# ---------- Default region/type from any top picks (optional) ----------
default_region = "Unknown"
default_type   = "Unknown"

if 'top3_by_type' in globals() and isinstance(top3_by_type, pd.DataFrame) and not top3_by_type.empty:
    top_rec = top3_by_type.iloc[0]
    wine_id = str(top_rec.get('id', "")).strip()
    default_type = str(top_rec.get('full_type', 'Unknown')) or 'Unknown'
    try:
        stock_df_ids = stock_df.copy()
        stock_df_ids['id'] = stock_df_ids['id'].astype(str)
        match_row = stock_df_ids.loc[stock_df_ids['id'] == wine_id, 'region_group']
        if not match_row.empty:
            default_region = str(match_row.iloc[0]) or "Unknown"
    except Exception:
        pass
else:
    print("⚠️ top3_by_type is empty — using 'Unknown' defaults for region/type.")

# Build seg_df filling missing values with defaults (but not overwriting existing)
seg_df = client_pref_df.copy()
seg_df['region_group'] = seg_df.get('region_group', pd.Series(index=seg_df.index, dtype='object')).fillna(default_region)
seg_df['full_type']    = seg_df.get('full_type',    pd.Series(index=seg_df.index, dtype='object')).fillna(default_type)

# ---------- Load UI filters (file -> env fallback -> defaults) ----------
filters_path = Path("notebooks") / "filters.json"
filters = {}
try:
    if filters_path.exists():
        filters = json.loads(filters_path.read_text(encoding="utf-8")) or {}
    elif os.getenv("FILTER_INPUTS"):
        filters = json.loads(os.getenv("FILTER_INPUTS"))
except Exception as e:
    print(f"⚠️ Failed to read filters.json/env: {e}")
finally:
    if not filters:
        filters = {
            "last_stock": False,
            "seasonality_boost": False,
            "wine_type": None,
            "bottle_size": None,
            "price_tier_bucket": "",
            "price_tiers": [],
            "loyalty_levels": []
        }

print("🔍 Effective UI filters:\n", json.dumps(filters, indent=2))

# ---------- Normalize stock needed columns ----------
stock_df = stock_df.copy()
stock_df['id']    = stock_df.get('id', "").astype(str)
stock_df['stock'] = pd.to_numeric(stock_df.get('stock', 0), errors='coerce').fillna(0).astype(int)
stock_df['price_tier'] = stock_df.get('price_tier', "").map(canon_tier_name)
stock_df['price_tier_id'] = stock_df['price_tier'].map(tier_id_from_name)

if 'bottle_size_ml' not in stock_df.columns:
    if 'size' in stock_df.columns:
        stock_df['bottle_size_ml'] = stock_df['size'].apply(to_ml)
    elif 'size_cl' in stock_df.columns:
        stock_df['bottle_size_ml'] = pd.to_numeric(stock_df['size_cl'], errors='coerce') * 10
    else:
        stock_df['bottle_size_ml'] = np.nan

stock_df = ensure_cols(stock_df, {'occasion': 'Unknown'})

# ---------- Apply global UI filters ----------
_selected_tiers = set()
if filters.get("price_tiers"):
    _selected_tiers.update([canon_tier_name(t) for t in filters["price_tiers"] if t])
if filters.get("price_tier_bucket"):
    _selected_tiers.add(canon_tier_name(filters["price_tier_bucket"]))

if _selected_tiers:
    stock_df = stock_df[stock_df['price_tier'].isin(_selected_tiers)].copy()

_selected_loyalties = {str(x).strip().lower() for x in (filters.get("loyalty_levels") or []) if str(x).strip()}
if _selected_loyalties and 'loyalty_level' in seg_df.columns:
    seg_df = seg_df[seg_df['loyalty_level'].str.lower().isin(_selected_loyalties)].copy()

last_stock_flag    = bool(filters.get("last_stock", False))
seasonality_flag   = bool(filters.get("seasonality_boost", False))
selected_type      = filters.get("wine_type", None)
selected_size_raw  = filters.get("bottle_size", None)
try:
    selected_size_ml = int(selected_size_raw) if (selected_size_raw is not None and str(selected_size_raw).isdigit()) else None
except Exception:
    selected_size_ml = None

# ---------- Weekday policies ----------
WEEKDAY_OCCASION = {
    "Monday": "Casual", "Tuesday": "Casual", "Wednesday": "Dinner",
    "Thursday": "Dinner", "Friday": "Party", "Saturday": "Gifting", "Sunday": "Dinner"
}
WEEKDAY_TIERS = {
    "Monday":   ["Budget", "Mid-range", "Premium"],
    "Tuesday":  ["Mid-range", "Premium"],
    "Wednesday":["Premium", "Luxury"],
    "Thursday": ["Premium", "Luxury"],
    "Friday":   ["Luxury", "Ultra Luxury"],
    "Saturday": ["Luxury", "Ultra Luxury"],
    "Sunday":   ["Budget", "Premium", "Luxury"],
}

# ---------- Selector ----------
def get_seasonal_wines_modular(stock_df, seg_df, last_stock_flag=False, seasonality=False,
                               selected_type=None, selected_size_ml=None, num_per_day=5):
    df = stock_df.copy()

    # Type filter
    if selected_type and str(selected_type).strip().lower() not in ("all", ""):
        df = df[df['full_type'].astype(str).str.casefold() == str(selected_type).casefold()]

    # Size filter (ml)
    if selected_size_ml is not None and not np.isnan(selected_size_ml):
        df = df[np.isclose(pd.to_numeric(df['bottle_size_ml'], errors='coerce'), selected_size_ml, equal_nan=False)]

    # Seasonality: within ~last year → next week window (based on OMT last offer date)
    if seasonality:
        last_year = datetime.today() - timedelta(days=365)
        next_week = last_year + timedelta(days=7)
        df['OMT last offer date'] = pd.to_datetime(df.get('OMT last offer date', pd.NaT), errors='coerce')
        df = df[df['OMT last offer date'].between(last_year, next_week)]

    # Stock filters
    if last_stock_flag:
        df = df[df['stock'] < 10]
    df = df[df['stock'] >= 3]

    # Segment match score (region+type)
    seg = seg_df[['region_group','full_type']].dropna()
    def match_score(row):
        return int(((seg['region_group'] == row['region_group']) & (seg['full_type'] == row['full_type'])).sum()) if not seg.empty else 0
    df['segment_score'] = df.apply(match_score, axis=1) if not df.empty else 0

    ultra_cap = 2
    ultra_used = 0

    calendar = {}

    for day in DAYS:
        allowed_tiers = [canon_tier_name(t) for t in WEEKDAY_TIERS.get(day, [])]
        occasion = WEEKDAY_OCCASION.get(day, "Casual")

        pool = df.copy()

        # Step 1: match occasion + tier
        cand = pool[(pool['occasion'].astype(str).str.casefold() == occasion.casefold()) &
                    (pool['price_tier'].isin(allowed_tiers))]

        # Ultra cap per week
        if ultra_used >= ultra_cap:
            cand = cand[cand['price_tier'] != "Ultra Luxury"]

        sel = cand.sort_values(['segment_score','stock'], ascending=[False, False]).head(num_per_day)

        # Step 2: relax occasion
        if sel.empty:
            cand2 = pool[pool['price_tier'].isin(allowed_tiers)]
            if ultra_used >= ultra_cap:
                cand2 = cand2[cand2['price_tier'] != "Ultra Luxury"]
            sel = cand2.sort_values(['segment_score','stock'], ascending=[False, False]).head(num_per_day)

        # Step 3: seasonal fallback
        if sel.empty:
            seasonal_only = pool.copy()
            seasonal_only['OMT last offer date'] = pd.to_datetime(seasonal_only.get('OMT last offer date', pd.NaT), errors='coerce')
            last_year = datetime.today() - timedelta(days=365)
            next_week = last_year + timedelta(days=7)
            fallback = seasonal_only[seasonal_only['OMT last offer date'].between(last_year, next_week)]
            sel = fallback.sort_values('stock', ascending=False).head(num_per_day)

        # Step 4: top stocked fallback
        if sel.empty:
            sel = pool.sort_values('stock', ascending=False).head(num_per_day)

        # track ultra usage
        if not sel.empty and "Ultra Luxury" in sel['price_tier'].values:
            ultra_used = min(ultra_cap, ultra_used + (sel['price_tier'].eq("Ultra Luxury").sum()))

        calendar[day] = sel

    return calendar

# ---------- UI mapping ----------
def to_ui_item(row: pd.Series) -> dict:
    return {
        'id': str(row.get('id', '')),
        'wine': row.get('wine', 'Unknown'),
        'name': row.get('wine', 'Unknown'),
        'vintage': str(row.get('vintage', 'NV')) if pd.notna(row.get('vintage', '')) else 'NV',
        'full_type': row.get('full_type', 'Unknown'),
        'region_group': row.get('region_group', 'Unknown'),
        'stock': int(pd.to_numeric(row.get('stock', 0), errors='coerce') if row.get('stock', 0) is not None else 0),
        'price_tier': canon_tier_name(row.get('price_tier', '')),
        'price_tier_id': tier_id_from_name(row.get('price_tier', '')),
        'match_quality': row.get('match_quality', 'Auto'),
        'avg_cpi_score': float(pd.to_numeric(row.get('avg_cpi_score', 0), errors='coerce')) if pd.notna(row.get('avg_cpi_score', np.nan)) else 0.0,
        'locked': bool(row.get('locked', False))
    }

def get_ui_data(top_n_recs: pd.DataFrame, weekly_wines: dict, top_wine_counts: pd.DataFrame,
                client_prefs: pd.DataFrame, locked_df: pd.DataFrame | None = None):
    calendar_data = {d: [] for d in DAYS}

    # 1) Locked first
    if isinstance(locked_df, pd.DataFrame) and not locked_df.empty:
        for day in DAYS:
            for _, r in locked_df.loc[locked_df['day']==day].iterrows():
                calendar_data[day].append(to_ui_item(r))

    # 2) Auto picks (skip duplicates)
    for day, df_day in (weekly_wines or {}).items():
        if isinstance(df_day, pd.DataFrame) and not df_day.empty:
            have = {str(it.get("id","")) for it in calendar_data[day]}
            for _, r in df_day.iterrows():
                rid = str(r.get("id",""))
                if rid and rid not in have:
                    calendar_data[day].append(to_ui_item(r))
                    have.add(rid)

    # CPI preview (avg of attached)
    cpi_score = 0.0
    try:
        flat = [it.get("avg_cpi_score", 0) for arr in calendar_data.values() for it in arr]
        if flat:
            cpi_score = float(pd.to_numeric(pd.Series(flat), errors="coerce").fillna(0).mean())
    except Exception:
        pass

    return {
        'weekly_calendar': calendar_data,
        'cpi_score': round(cpi_score, 2),
        'client_prefs': client_prefs.to_dict('records'),
        'top_recommendations': []
    }

# ---------- Build weekly selection ----------
IRON_DATA_PATH = Path.home() / "OneDrive - AVU SA" / "AVU CPI Campaign" / "Puzzle_control_Reports" / "IRON_DATA"
IRON_DATA_PATH.mkdir(parents=True, exist_ok=True)

weekly_wines = get_seasonal_wines_modular(
    stock_df, seg_df,
    last_stock_flag=last_stock_flag,
    seasonality=seasonality_flag,
    selected_type=selected_type,
    selected_size_ml=selected_size_ml,
    num_per_day=NUM_SLOTS
)

ui_output_data = get_ui_data(
    top_n_recs=pd.DataFrame(), 
    weekly_wines=weekly_wines, 
    top_wine_counts=pd.DataFrame(), 
    client_prefs=client_pref_df, 
    locked_df=locked_df
)

# Normalize & cap to NUM_SLOTS (keeps locked-first order due to append sequence)
calendar_norm = {d: (ui_output_data['weekly_calendar'].get(d, []) or [])[:NUM_SLOTS] for d in DAYS}

# ---------- Write artifacts ----------
(IRON_DATA_PATH / "weekly_campaign_schedule.json").write_text(
    json.dumps(calendar_norm, indent=4), encoding="utf-8"
)
(IRON_DATA_PATH / f"weekly_campaign_schedule_{sel_year}_week_{sel_week}.json").write_text(
    json.dumps(calendar_norm, indent=4), encoding="utf-8"
)
# Legacy (week only)
(IRON_DATA_PATH / f"weekly_campaign_schedule_week_{sel_week}.json").write_text(
    json.dumps(calendar_norm, indent=4), encoding="utf-8"
)
# UI-preferred artifact
(IRON_DATA_PATH / f"weekly_campaign_schedule_{sel_year}_week_{sel_week}.ui.json").write_text(
    json.dumps({"weekly_calendar": calendar_norm}, indent=2), encoding="utf-8"
)

# PKLs
pd.to_pickle(calendar_norm, IRON_DATA_PATH / "weekly_campaign_schedule.pkl")
pd.to_pickle(calendar_norm, IRON_DATA_PATH / f"weekly_campaign_schedule_{sel_year}_week_{sel_week}.pkl")
pd.to_pickle(calendar_norm, IRON_DATA_PATH / f"weekly_campaign_schedule_week_{sel_week}.pkl")

# Leads (present if previous logic produced it; otherwise sane default)
leads_payload = ui_output_data.get("leads_campaigns", {"TueWed": [], "ThuFri": []})
(IRON_DATA_PATH / "leads_campaigns.json").write_text(json.dumps(leads_payload, indent=2), encoding="utf-8")
(IRON_DATA_PATH / f"leads_campaigns_{sel_year}_week_{sel_week}.json").write_text(json.dumps(leads_payload, indent=2), encoding="utf-8")

# Index (nested by year→week)
index_path = IRON_DATA_PATH / "schedule_index.json"
try:
    idx = json.loads(index_path.read_text(encoding="utf-8")) if index_path.exists() else {}
except Exception:
    idx = {}
idx.setdefault(str(sel_year), {})[str(sel_week)] = {
    "json": f"weekly_campaign_schedule_{sel_year}_week_{sel_week}.json",
    "json_ui": f"weekly_campaign_schedule_{sel_year}_week_{sel_week}.ui.json",
    "pkl": f"weekly_campaign_schedule_{sel_year}_week_{sel_week}.pkl",
    "leads_json": f"leads_campaigns_{sel_year}_week_{sel_week}.json",
    "updated_at": datetime.now().isoformat(timespec="seconds")
}
idx["_latest_year"] = str(sel_year)
idx["_latest_week"] = str(sel_week)
(IRON_DATA_PATH / "schedule_index.json").write_text(json.dumps(idx, indent=2), encoding="utf-8")

print(f"✅ Saved year+week UI files for {sel_year}-W{sel_week} in {IRON_DATA_PATH}")


⚠️ top3_by_type is empty — using 'Unknown' defaults for region/type.
🔍 Effective UI filters:
 {
  "loyalty": "all",
  "wine_type": null,
  "bottle_size": 750,
  "price_tier_bucket": "",
  "last_stock": true,
  "last_stock_threshold": 10,
  "seasonality_boost": false,
  "style": "default",
  "calendar_day": null
}


✅ Saved year+week UI files for 2025-W36 in C:\Users\Marco.Africani\OneDrive - AVU SA\AVU CPI Campaign\Puzzle_control_Reports\IRON_DATA


In [10]:
# --- CELL 9: Build weekly "Leads" campaigns (Tue–Wed & Thu–Fri) ---
from pathlib import Path
import pandas as pd
import numpy as np
from datetime import datetime
import json

# Ensure IRON_DATA_PATH exists (fallback to OUTPUT_PATH or default OneDrive path)
if 'IRON_DATA_PATH' not in globals():
    if 'OUTPUT_PATH' in globals():
        IRON_DATA_PATH = OUTPUT_PATH
    else:
        IRON_DATA_PATH = Path.home() / "OneDrive - AVU SA" / "AVU CPI Campaign" / "Puzzle_control_Reports" / "IRON_DATA"
IRON_DATA_PATH.mkdir(parents=True, exist_ok=True)

def _canon_tier_name(x: str) -> str:
    s = str(x or "").strip().lower().replace("-", " ").replace("_", " ")
    if not s: return ""
    if "ultra" in s: return "Ultra Luxury"
    if "luxury" in s: return "Luxury"
    if "premium" in s: return "Premium"
    if "mid" in s: return "Mid-range"
    if "budget" in s or "<" in s or "cheap" in s: return "Budget"
    return s.title()

def _load_inventory(iron_path: Path) -> pd.DataFrame:
    # Prefer the most enriched inventory if present
    candidates = [
        iron_path / "stock_df_with_seasonality.pkl",
        iron_path / "stock_df_final.pkl",
    ]
    src = next((p for p in candidates if p.exists()), None)
    if not src:
        return pd.DataFrame(columns=["id","wine","vintage","full_type","region_group","stock","price_tier","avg_cpi_score"])
    df = pd.read_pickle(src)

    # Normalize columns we rely on
    for c in ("id","wine","vintage","full_type","region_group","stock","price_tier","price","avg_cpi_score"):
        if c not in df.columns:
            df[c] = np.nan

    df["id"] = df["id"].astype("string").fillna("").str.replace(r"\.0$", "", regex=True).str.strip()
    df["wine"] = df["wine"].astype("string").fillna("").str.strip()
    df["vintage"] = df["vintage"].astype("string").fillna("").str.strip().replace({"": "NV","nan":"NV"})
    df["full_type"] = df["full_type"].astype("string").fillna("").str.strip()
    df["region_group"] = df["region_group"].astype("string").fillna("").str.strip()
    df["stock"] = pd.to_numeric(df["stock"], errors="coerce").fillna(0).astype(int)

    # Price tier: prefer provided; else derive from price if present
    df["price_tier"] = df["price_tier"].astype("string").fillna("").map(_canon_tier_name)
    if df["price_tier"].eq("").any() and "price" in df.columns:
        price_num = pd.to_numeric(df["price"], errors="coerce")
        def _bucket(p):
            if pd.isna(p): return ""
            if p < 50: return "Budget"
            if p < 100: return "Mid-range"
            if p < 200: return "Premium"
            if p < 500: return "Luxury"
            return "Ultra Luxury"
        mask = df["price_tier"].eq("")
        df.loc[mask, "price_tier"] = price_num.map(_bucket)

    # keep only rows with something to offer
    df = df[df["stock"] > 0].copy()
    return df

def _attach_last_campaign(inv: pd.DataFrame, history: pd.DataFrame | None) -> pd.DataFrame:
    """
    Attach last_campaign_date to inventory rows. Uses history_df when provided;
    otherwise falls back to OUTPUT/IRON campaign_index.json if available.
    """
    inv = inv.copy()
    inv["last_campaign_date"] = pd.NaT

    # 1) If a rich history_df is available
    if history is not None and not history.empty and {"id","wine","vintage","schedule_dt"}.issubset(history.columns):
        hist = history.copy()
        hist["id"] = hist["id"].astype("string").fillna("").str.replace(r"\.0$","",regex=True).str.strip()
        hist["wine"] = hist["wine"].astype("string").fillna("").str.strip()
        hist["vintage"] = hist["vintage"].astype("string").fillna("").str.strip().replace({"": "NV","nan":"NV"})
        hist["schedule_dt"] = pd.to_datetime(hist["schedule_dt"], errors="coerce")

        last_seen = (
            hist.dropna(subset=["schedule_dt"])
                .groupby(["id","wine","vintage"], dropna=False)["schedule_dt"]
                .max()
                .reset_index()
                .rename(columns={"schedule_dt":"last_campaign_date"})
        )
        return inv.merge(last_seen, how="left", on=["id","wine","vintage"])

    # 2) Fallback: campaign_index.json (fast, id-based)
    idx_path = IRON_DATA_PATH / "campaign_index.json"
    if not idx_path.exists() and 'OUTPUT_PATH' in globals():
        alt = OUTPUT_PATH / "campaign_index.json"
        if alt.exists():
            idx_path = alt

    if idx_path.exists():
        try:
            ci = json.loads(idx_path.read_text(encoding="utf-8"))
            by_id = ci.get("by_id", {})
            id_map = {str(k): v.get("last_campaign_date") for k, v in by_id.items()}
            inv["last_campaign_date"] = inv["id"].map(id_map)
            inv["last_campaign_date"] = pd.to_datetime(inv["last_campaign_date"], errors="coerce")
        except Exception:
            pass

    return inv

def _pick_one(df, sort_cols, asc):
    if df.empty: return pd.DataFrame(columns=df.columns)
    return df.sort_values(sort_cols, ascending=asc, na_position="last").head(1)

def _pick_varied(df, used_ids, k=2):
    pool = df[~df["id"].isin(used_ids)].copy()
    picks = []

    # prefer different full_type across picks if possible
    type_order = ["Sparkling","White","Rosé","Rose","Red","Dessert"]
    for t in type_order:
        cand = pool[pool["full_type"].str.contains(t, case=False, na=False)]
        got = _pick_one(cand, ["stock","last_campaign_date"], [False, True])
        if not got.empty:
            picks.append(got)
            pool = pool[~pool["id"].isin(got["id"])]
        if len(picks) >= k:
            break

    # fallback if we didn’t get enough
    if len(picks) < k and not pool.empty:
        extra = pool.sort_values(["stock","last_campaign_date"], ascending=[False, True], na_position="last").head(k - len(picks))
        if not extra.empty:
            picks.append(extra)

    return pd.concat(picks, ignore_index=True) if picks else pd.DataFrame(columns=df.columns)

def build_weekly_leads(iron_path: Path, history_df: pd.DataFrame | None) -> dict:
    inv = _load_inventory(iron_path)
    inv = _attach_last_campaign(inv, history_df)

    # 1) Budget (high stock)
    budget = inv[inv["price_tier"].eq("Budget")]
    lead_budget = _pick_one(budget, ["stock","last_campaign_date"], [False, True])

    # 2) Ultra Luxury (low stock)
    ultra = inv[inv["price_tier"].eq("Ultra Luxury")]
    lead_ultra = _pick_one(ultra, ["stock","last_campaign_date"], [True, True])

    # 3–4) Two varied items (not already chosen)
    used_ids = pd.concat([lead_budget, lead_ultra])["id"].astype(str).tolist() if not pd.concat([lead_budget, lead_ultra]).empty else []
    varied = _pick_varied(inv, used_ids, k=2)

    leads_df = pd.concat([lead_budget, lead_ultra, varied], ignore_index=True)
    if leads_df.empty:
        return {"TueWed": [], "ThuFri": []}

    def _row_to_item(r):
        return {
            "id": str(r.get("id","")),
            "wine": r.get("wine") or "Unknown",
            "name": r.get("wine") or "Unknown",
            "vintage": str(r.get("vintage","NV")) if str(r.get("vintage","")).strip() else "NV",
            "full_type": r.get("full_type") or "Unknown",
            "region_group": r.get("region_group") or "Unknown",
            "stock": int(pd.to_numeric(r.get("stock", 0), errors="coerce") or 0),
            "price_tier": r.get("price_tier") or "",
            "match_quality": "Lead",
            "avg_cpi_score": float(pd.to_numeric(r.get("avg_cpi_score", 0), errors="coerce") or 0),
            "last_campaign_date": (
                r.get("last_campaign_date").isoformat()
                if pd.notna(r.get("last_campaign_date")) else None
            ),
            # UI helpers:
            "locked": False,
            "span_days": 2,             # 2-day “Leads” box
            "campaign_tag": "leads"
        }

    items = [ _row_to_item(r) for _, r in leads_df.iterrows() ]
    # Same set in both boxes (simple + consistent)
    return {"TueWed": items, "ThuFri": items}

# Get history_df if available; otherwise pass None (we’ll fall back to campaign_index.json)
_hist = history_df if 'history_df' in globals() else None
try:
    leads_campaigns = build_weekly_leads(IRON_DATA_PATH, _hist)
except Exception as e:
    print(f"⚠️ build_weekly_leads failed: {e}")
    leads_campaigns = {"TueWed": [], "ThuFri": []}

# Persist year+week leads alongside generic (for fast API fetches)
try:
    y, w = int(calendar_year), int(week_number)
except Exception:
    y, w = datetime.now().year, datetime.now().isocalendar().week

lp = IRON_DATA_PATH
lp.mkdir(parents=True, exist_ok=True)
(lp / "leads_campaigns.json").write_text(json.dumps(leads_campaigns, indent=2), encoding="utf-8")
(lp / f"leads_campaigns_{y}_week_{w}.json").write_text(json.dumps(leads_campaigns, indent=2), encoding="utf-8")
print(f"📣 Leads saved for {y}-W{w}")

# Attach to in-memory UI bundle if present
if "ui_output_data" not in locals():
    ui_output_data = {}
ui_output_data["leads_campaigns"] = leads_campaigns

# (Removed undefined atomic_write_* calls and the incorrect 'leads_payload' reference.)


⚠️ build_weekly_leads failed: 'last_campaign_date'
📣 Leads saved for 2025-W36


In [11]:
# --- CELL 10: Flask handoff & week outputs (atomic + robust) ---
import os
import json
from datetime import datetime, timezone, timedelta
from pathlib import Path
import pandas as pd
import numpy as np

# === Safety Check ===
if "ui_output_data" not in locals():
    raise RuntimeError("❌ ui_output_data not found. Make sure the previous cell populated it.")

# === Status bridge (prefer package import; fallback to none) ===
try:
    from utils.notebook_status import update_status  # when run by Flask
except Exception:
    try:
        from notebook_status import update_status     # when run locally
    except Exception as e:
        print(f"⚠️ update_status unavailable: {e}")
        update_status = None

# === Paths / Week ===
IRON_DATA_PATH = Path(
    globals().get("OUTPUT_PATH")
    or globals().get("output_path")
    or os.getenv("OUTPUT_PATH")
    or os.getenv("IRON_DATA")
    or (Path.home() / "OneDrive - AVU SA" / "AVU CPI Campaign" / "Puzzle_control_Reports" / "IRON_DATA")
)
IRON_DATA_PATH.mkdir(parents=True, exist_ok=True)

try:
    week_number = int(globals().get("week_number", os.getenv("WEEK_NUMBER", datetime.now().isocalendar().week)))
except Exception:
    week_number = datetime.now().isocalendar().week

# === Helpers ===
DAYS = ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]

TIER_MAP_NAME = {
    "budget":"Budget","entry":"Budget",
    "mid":"Mid-range","mid_range":"Mid-range","mid-range":"Mid-range",
    "premium":"Premium",
    "luxury":"Luxury",
    "ultra":"Ultra Luxury","ultra_luxury":"Ultra Luxury","ultra-luxury":"Ultra Luxury","ultra luxury":"Ultra Luxury",
}
TIER_MAP_ID = {"Budget":"budget","Mid-range":"mid","Premium":"premium","Luxury":"luxury","Ultra Luxury":"ultra"}

def canon_tier_name(x: str) -> str:
    s = str(x or "").strip()
    if not s: return ""
    key = s.lower().replace("-", "_").replace(" ", "_")
    return TIER_MAP_NAME.get(key, s)

def tier_id_from_name(name: str) -> str:
    return TIER_MAP_ID.get(canon_tier_name(name), "")

def _to_int(x, default=0):
    try:
        return int(float(x))
    except Exception:
        try:
            return int(pd.to_numeric(x, errors="coerce").fillna(default))
        except Exception:
            return default

def _now_iso():
    return datetime.now(timezone.utc).isoformat(timespec="seconds")

def atomic_write_text(path: Path, text: str):
    tmp = path.with_suffix(path.suffix + ".tmp")
    tmp.write_text(text, encoding="utf-8")
    os.replace(tmp, path)

def atomic_write_pkl(path: Path, obj):
    tmp = path.with_suffix(path.suffix + ".tmp")
    pd.to_pickle(obj, tmp)
    os.replace(tmp, path)

def ensure_calendar_schema(calendar: dict) -> dict:
    """Guarantee days exist, coerce items and cap to 5 per day."""
    out = {d: [] for d in DAYS}
    if not isinstance(calendar, dict):
        return out
    for day in DAYS:
        items = calendar.get(day, []) or []
        norm_items = []
        for it in items:
            it = it or {}
            price_tier = canon_tier_name(it.get("price_tier", it.get("price_tier_bucket", "")))
            norm_items.append({
                "id":              str(it.get("id", "")),
                "wine":            it.get("wine") or it.get("name") or "Unknown",
                "name":            it.get("name") or it.get("wine") or "Unknown",
                "vintage":         str(it.get("vintage", "NV")) if str(it.get("vintage","")).strip() else "NV",
                "full_type":       it.get("full_type", "Unknown"),
                "region_group":    it.get("region_group", "Unknown"),
                "stock":           _to_int(it.get("stock", it.get("stock_count", 0))),
                "price_tier":      price_tier,
                "price_tier_id":   tier_id_from_name(price_tier),
                "match_quality":   it.get("match_quality", "Auto"),
                "avg_cpi_score":   float(pd.to_numeric(it.get("avg_cpi_score", it.get("cpi_score", 0)), errors="coerce") or 0),
                "locked":          bool(it.get("locked", False)),
            })
        out[day] = norm_items[:5]
    return out

# === Normalize payload from Cell 8/9 ===
calendar_raw = ui_output_data.get("weekly_calendar", {})
calendar_norm = ensure_calendar_schema(calendar_raw)

# Leads payload produced in Cell 9
leads_payload = ui_output_data.get("leads_campaigns", {"TueWed": [], "ThuFri": []})

# Optional summary for status callback
summary = {
    "week_number": week_number,
    "updated_at": _now_iso(),
    "days": {d: len(calendar_norm.get(d, [])) for d in DAYS},
    "cpi_score": ui_output_data.get("cpi_score", None),
    "top_recommendations": ui_output_data.get("top_recommendations", []),
    "leads_counts": {k: len(v or []) for k, v in (leads_payload or {}).items()}
}

# === Save (atomic) ===
json_generic = IRON_DATA_PATH / "weekly_campaign_schedule.json"
pkl_generic  = IRON_DATA_PATH / "weekly_campaign_schedule.pkl"
json_week    = IRON_DATA_PATH / f"weekly_campaign_schedule_week_{week_number}.json"
pkl_week     = IRON_DATA_PATH / f"weekly_campaign_schedule_week_{week_number}.pkl"

# Dedicated leads files
leads_json_generic = IRON_DATA_PATH / "leads_campaigns.json"
leads_json_week    = IRON_DATA_PATH / f"leads_campaigns_week_{week_number}.json"

index_path   = IRON_DATA_PATH / "schedule_index.json"

try:
    # Write calendar artifacts
    atomic_write_text(json_generic, json.dumps(calendar_norm, indent=4))
    atomic_write_text(json_week,    json.dumps(calendar_norm, indent=4))
    atomic_write_pkl(pkl_generic, calendar_norm)
    atomic_write_pkl(pkl_week,    calendar_norm)

    # Write leads artifacts
    atomic_write_text(leads_json_generic, json.dumps(leads_payload, indent=2))
    atomic_write_text(leads_json_week,    json.dumps(leads_payload, indent=2))

    # Maintain a simple index for the server
    try:
        idx = json.loads(index_path.read_text(encoding="utf-8")) if index_path.exists() else {}
    except Exception:
        idx = {}
    idx[str(week_number)] = {
        "json": json_week.name,
        "pkl":  pkl_week.name,
        "leads_json": leads_json_week.name,
        "updated_at": summary["updated_at"]
    }
    idx["_latest"] = str(week_number)
    atomic_write_text(index_path, json.dumps(idx, indent=2))

    print(f"✅ UI JSON and PKL saved atomically for week {week_number}")
    print("📅 Days in calendar:", list(calendar_norm.keys()))
    print("🧮 Slots per day:", {d: len(calendar_norm[d]) for d in DAYS})
    print("🎯 CPI score:", ui_output_data.get('cpi_score', 'N/A'))
    print("📣 Leads (counts):", summary["leads_counts"])

except Exception as e:
    print(f"❌ An error occurred while saving UI files: {e}")

# === Report back to Flask UI (if available) ===
if callable(update_status):
    try:
        _nb_name = globals().get("NOTEBOOK_NAME") or os.getenv("NOTEBOOK_NAME") or "AVU_schedule_only.ipynb"
        update_status(
            progress=100,
            message="✅ Notebook finished.",
            state="completed",
            done=True,
            notebook=_nb_name,
            meta=summary
        )
    except Exception as e:
        print(f"⚠️ update_status failed: {e}")


✅ UI JSON and PKL saved atomically for week 36
📅 Days in calendar: ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
🧮 Slots per day: {'Monday': 5, 'Tuesday': 5, 'Wednesday': 5, 'Thursday': 5, 'Friday': 5, 'Saturday': 5, 'Sunday': 5}
🎯 CPI score: 0.06
📣 Leads (counts): {'TueWed': 0, 'ThuFri': 0}
⚠️ update_status failed: update_status() got an unexpected keyword argument 'progress'


In [12]:
# --- CELL 11: Generate a file readable by Power BI (from normalized stock) 
#--- then PowerBI visualizion will be on the UI
import os
from datetime import datetime
from pathlib import Path
import numpy as np
import pandas as pd

# 1) Get the normalized stock frame
try:
    df = stock_df.copy()  # from Cell 3
except NameError:
    # Fallback: read from the artifact saved in Cell 3
    df_path = OUTPUT_PATH / "stock_df_final.pkl"
    if not df_path.exists():
        raise FileNotFoundError(f"❌ Could not find {df_path}. Run Cell 3 first.")
    df = pd.read_pickle(df_path)

# 2) Ensure required columns exist
for col in ["price_tier", "CHF Price", "vintage", "id", "wine"]:
    if col not in df.columns:
        df[col] = np.nan

# Normalize numeric CHF price
df["CHF Price"] = pd.to_numeric(df["CHF Price"], errors="coerce")

# 3) Canonicalize price tiers + add stable ID
def price_tier_from_price(price):
    try:
        p = float(price)
    except Exception:
        return None
    if p < 50:   return "Budget"
    if p < 100:  return "Mid-range"
    if p < 200:  return "Premium"
    if p < 500:  return "Luxury"
    return "Ultra Luxury"

def canon_tier_name(x):
    s = str(x or "").strip().lower()
    if not s: return ""
    if "ultra" in s: return "Ultra Luxury"
    if "luxury" in s: return "Luxury"
    if "premium" in s: return "Premium"
    if "mid" in s: return "Mid-range"
    if "budget" in s: return "Budget"
    return "Unknown"

def tier_id(name):
    m = {
        "Budget":"budget",
        "Mid-range":"mid",
        "Premium":"premium",
        "Luxury":"luxury",
        "Ultra Luxury":"ultra",
        "Unknown":""
    }
    return m.get(name, "")

# Fill missing/blank tiers from price, then canonicalize
df["price_tier"] = df["price_tier"].where(df["price_tier"].notna() & (df["price_tier"].astype(str).str.strip()!=""))
df.loc[df["price_tier"].isna() | (df["price_tier"].astype(str).str.strip()==""), "price_tier"] = df["CHF Price"].apply(price_tier_from_price)
df["Cleaned_Price_Tier"] = df["price_tier"].apply(canon_tier_name)
df["price_tier_id"] = df["Cleaned_Price_Tier"].apply(tier_id)

# 4) Vintage grouping (Power BI-friendly labels)
def classify_vintage_group(vint):
    try:
        s = str(vint).strip().upper()
        if s == "NV": return "Non-Vintage"
        y = int(s)
        cy = datetime.now().year
        if y == cy:       return "Current Vintage"
        if y == cy - 1:   return "En Primeur"
        if cy - y <= 3:   return "Young Wines"
        if cy - y <= 8:   return "Mature Wines"
        if cy - y > 15:   return "Old Wines"
        return "Unknown"
    except Exception:
        return "Unknown"

df["Vintage_Group"] = df["vintage"].apply(classify_vintage_group)

# 5) Order within each tier for triangular/scatter layout
#    Sort by tier (alphabetical works with our canonical names) then by CHF price desc
df = df.sort_values(["Cleaned_Price_Tier", "CHF Price"], ascending=[True, False]).reset_index(drop=True)

# Tier-local index
df["Tier_Index"] = df.groupby("Cleaned_Price_Tier").cumcount() + 1

# Row of triangular packing (n-th triangular number inverse)
def compute_row(idx):
    # idx is 1-based
    return int((8 * idx + 1) ** 0.5 - 1) // 2

df["Row_Number"] = df["Tier_Index"].apply(compute_row)

# Vertical coordinate (top-down)
df["Arrow_Y"] = -df["Row_Number"]

# Horizontal coordinate centered within each row
df["Arrow_X_raw"] = (
    df["Tier_Index"]
    - (df["Row_Number"] * (df["Row_Number"] + 1) // 2)
    - (df["Row_Number"] / 2.0)
)

# Horizontal offsets per tier (keeps tiers separated left→right)
tier_offsets = {
    "Budget": 0,
    "Mid-range": 10,
    "Premium": 20,
    "Luxury": 30,
    "Ultra Luxury": 40
}
df["Tier_X_Offset"] = df["Cleaned_Price_Tier"].map(tier_offsets).fillna(0)
df["Arrow_X"] = df["Arrow_X_raw"] + df["Tier_X_Offset"]

# 6) Select nice columns for Power BI (keep original too)
cols = [
    "id","wine","vintage","Vintage_Group",
    "CHF Price","stock","region_group","full_type",
    "price_tier","Cleaned_Price_Tier","price_tier_id",
    "Tier_Index","Row_Number","Arrow_X","Arrow_Y"
]
present_cols = [c for c in cols if c in df.columns]
out_df = df[present_cols].copy()

# 7) Save to IRON_DATA/OUTPUT_PATH so Power BI can pick it up
out_xlsx = OUTPUT_PATH / "powerbi_wine_arrow_layout.xlsx"
out_csv  = OUTPUT_PATH / "powerbi_wine_arrow_layout.csv"

out_df.to_excel(out_xlsx, index=False)
out_df.to_csv(out_csv, index=False, encoding="utf-8")

print("✅ Power BI layout files saved:")
print("   •", out_xlsx)
print("   •", out_csv)
print("🧭 Rows:", len(out_df), "| Columns:", len(out_df.columns))


✅ Power BI layout files saved:
   • C:\Users\Marco.Africani\OneDrive - AVU SA\AVU CPI Campaign\Puzzle_control_Reports\IRON_DATA\powerbi_wine_arrow_layout.xlsx
   • C:\Users\Marco.Africani\OneDrive - AVU SA\AVU CPI Campaign\Puzzle_control_Reports\IRON_DATA\powerbi_wine_arrow_layout.csv
🧭 Rows: 731 | Columns: 15
